## PART 1: Data Collection

##### 1.1: Fetch US stock price data (7,400 entries)

In [ ]:
# step1_fetch_stock_prices.py

"""
Fetch US stock price data using akshare.
Goal: 10 stocks, past two years.
"""

import akshare as ak
import pandas as pd
from datetime import datetime, timedelta
import time
import os

# Configuration
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'NVDA', 'TSLA', 'JPM', 'BAC', 'GS', 'WMT', 'DIS']
START_DATE = '2023-01-01'
END_DATE = '2025-12-13'
OUTPUT_DIR = '.'  # current folder

def fetch_single_stock(ticker, start_date, end_date):
    """
    Fetch historical price data for a single stock.
    """
    try:
        print(f"Fetching data for {ticker}...")

        # Fetch US daily data using akshare
        df = ak.stock_us_daily(symbol=ticker, adjust="qfq")

        # Convert date format
        df['date'] = pd. to_datetime(df['date'])

        # Filter by date range
        df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]

        # Add ticker column
        df['ticker'] = ticker

        # Reorder columns
        df = df[['ticker', 'date', 'open', 'high', 'low', 'close', 'volume']]

        print(f"✅ {ticker}: fetched {len(df)} records")
        return df

    except Exception as e:
        print(f"❌ {ticker} fetch failed: {e}")
        return None

def fetch_all_stocks(tickers, start_date, end_date):
    """
    Fetch all stock data in batch.
    """
    all_data = []

    for ticker in tickers:
        df = fetch_single_stock(ticker, start_date, end_date)
        if df is not None:
            all_data.append(df)

        # Avoid too frequent requests, sleep a bit
        time.sleep(2)

    # Combine all data
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        return combined_df
    else:
        return None

def save_data(df, output_dir):
    """
    Save data to CSV files (current folder).
    """
    # Save combined data to current folder
    combined_file = os.path. join(output_dir, 'all_stocks_prices.csv')
    df.to_csv(combined_file, index=False, encoding='utf-8-sig')
    print(f"\n✅ Combined data saved to: {combined_file}")

    # Also save each stock's data to separate files in current folder
    for ticker in df['ticker'].unique():
        ticker_df = df[df['ticker'] == ticker]
        ticker_file = os. path.join(output_dir, f'{ticker}_prices.csv')
        ticker_df.to_csv(ticker_file, index=False, encoding='utf-8-sig')
        print(f"   - {ticker}: {ticker_file}")

def main():
    """
    Main function
    """
    print("="*60)
    print("📊 Start fetching US stock price data")
    print("="*60)
    print(f"Stock list: {', '.join(TICKERS)}")
    print(f"Date range: {START_DATE} to {END_DATE}")
    print(f"Save location: current folder")
    print("="*60 + "\n")

    # Fetch data
    df = fetch_all_stocks(TICKERS, START_DATE, END_DATE)

    if df is not None:
        # Show statistics
        print("\n" + "="*60)
        print("📈 Data statistics")
        print("="*60)
        print(f"Total records: {len(df)}")
        print(f"Number of stocks: {df['ticker'].nunique()}")
        print(f"Date range:  {df['date'].min()} to {df['date'].max()}")
        print("\nRecords per stock:")
        print(df['ticker'].value_counts().sort_index())

        # Save data
        save_data(df, OUTPUT_DIR)

        # Show preview of first rows
        print("\n" + "="*60)
        print("📋 Data preview (first 10 rows)")
        print("="*60)
        print(df.head(10))

        print("\n✅ Step 1 complete! Stock price data fetched.")
        print(f"\n📁 Generated files in current folder:")
        print(f"   - all_stocks_prices.csv (all stocks combined)")
        print(f"   - AAPL_prices.csv, MSFT_prices.csv, etc. (individual stocks)")

    else:
        print("\n❌ Data fetch failed, please check your network connection or akshare version.")

if __name__ == "__main__":
    main()

📊 开始获取美股价格数据
股票列表: AAPL, MSFT, GOOGL, NVDA, TSLA, JPM, BAC, GS, WMT, DIS
日期范围: 2023-01-01 到 2025-12-13
保存位置: 当前文件夹

正在获取 AAPL 的数据...
✅ AAPL:  获取了 740 条数据
正在获取 MSFT 的数据...
✅ MSFT:  获取了 740 条数据
正在获取 GOOGL 的数据...
✅ GOOGL:  获取了 740 条数据
正在获取 NVDA 的数据...
✅ NVDA:  获取了 740 条数据
正在获取 TSLA 的数据...
✅ TSLA:  获取了 740 条数据
正在获取 JPM 的数据...
✅ JPM:  获取了 740 条数据
正在获取 BAC 的数据...
✅ BAC:  获取了 740 条数据
正在获取 GS 的数据...
✅ GS:  获取了 740 条数据
正在获取 WMT 的数据...
✅ WMT:  获取了 740 条数据
正在获取 DIS 的数据...
✅ DIS:  获取了 740 条数据

📈 数据统计
总记录数: 7400
股票数量: 10
日期范围:  2023-01-03 00:00:00 到 2025-12-12 00:00:00

各股票数据量:
ticker
AAPL     740
BAC      740
DIS      740
GOOGL    740
GS       740
JPM      740
MSFT     740
NVDA     740
TSLA     740
WMT      740
Name: count, dtype: int64

✅ 合并数据已保存到: .\all_stocks_prices.csv
   - AAPL: .\AAPL_prices.csv
   - MSFT: .\MSFT_prices.csv
   - GOOGL: .\GOOGL_prices.csv
   - NVDA: .\NVDA_prices.csv
   - TSLA: .\TSLA_prices.csv
   - JPM: .\JPM_prices.csv
   - BAC: .\BAC_prices.csv
   - GS: .\GS_prices.csv
  

##### 1.2: Fetch US stock news from Finviz

In [ ]:
# step2_scrape_finviz_news.py

"""
Scrape US stock news from Finviz.
Advantages: No login required, weak anti-scraping, clear data structure.
Goal: 10 stocks × 500 news = 5,000 items.
"""

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from datetime import datetime, timedelta
import re

# Configuration
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'NVDA', 'TSLA', 'JPM', 'BAC', 'GS', 'WMT', 'DIS']
OUTPUT_FILE = 'finviz_news_data.csv'

# Request headers (simulate real browser)
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}

def parse_finviz_date(date_str):
    """
    Parse Finviz date format.
    Example: 'Dec-13-24 09:30AM' or '09:30AM'
    """
    try:
        # If only time, it's today
        if re.match(r'^\d{1,2}:\d{2}(AM|PM)$', date_str):
            return datetime.now().strftime('%Y-%m-%d')

        # Parse full date
        # Format: Dec-13-24
        date_match = re.match(r'([A-Za-z]{3})-(\d{2})-(\d{2})', date_str)
        if date_match:
            month_str, day, year = date_match.groups()
            # Convert month
            months = {
                'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
                'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
                'Sep': '09', 'Oct': '10', 'Nov':  '11', 'Dec': '12'
            }
            month = months. get(month_str, '01')
            year = f"20{year}"
            return f"{year}-{month}-{day}"

        return datetime.now().strftime('%Y-%m-%d')
    except:
        return datetime.now().strftime('%Y-%m-%d')

def fetch_finviz_news(ticker):
    """
    Fetch news for a single stock from Finviz.
    """
    news_list = []

    try:
        url = f"https://finviz.com/quote.ashx?t={ticker}"
        print(f"  📡 Accessing:  {url}")

        response = requests.get(url, headers=HEADERS, timeout=15)
        
        if response. status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            # Finviz news table
            news_table = soup.find('table', {'id':  'news-table'})

            if news_table:
                rows = news_table.find_all('tr')

                current_date = None

                for row in rows:
                    try:
                        cells = row.find_all('td')

                        if len(cells) >= 2:
                            # First column: date/time
                            date_cell = cells[0]. text.strip()

                            # Second column: news title and link
                            title_elem = cells[1].find('a')

                            if title_elem:
                                title = title_elem. text.strip()
                                link = title_elem. get('href', '')

                                # Parse date
                                parsed_date = parse_finviz_date(date_cell)

                                # Extract source
                                source_elem = cells[1].find('span')
                                source = source_elem.text.strip() if source_elem else 'Unknown'
                                
                                news_list. append({
                                    'ticker': ticker,
                                    'date': parsed_date,
                                    'time': date_cell,
                                    'title': title,
                                    'url': link,
                                    'source': source,
                                    'platform': 'finviz'
                                })
                    except Exception as e:
                        continue

                print(f"  ✅ {ticker}:  Successfully fetched {len(news_list)} news items")
            else:
                print(f"  ⚠️ {ticker}: News table not found")
        else:
            print(f"  ❌ {ticker}: HTTP {response.status_code}")

    except requests.exceptions. Timeout:
        print(f"  ❌ {ticker}:  Request timed out")
    except Exception as e:
        print(f"  ❌ {ticker}: {str(e)}")

    return news_list

def fetch_all_tickers(tickers):
    """
    Batch fetch news for all stocks.
    """
    all_news = []
    total = len(tickers)

    print(f"\n{'='*60}")
    print(f"Start fetching news for {total} stocks")
    print(f"{'='*60}\n")

    for idx, ticker in enumerate(tickers, 1):
        print(f"[{idx}/{total}] Processing:  {ticker}")
        print("-" * 40)

        # Fetch news
        news = fetch_finviz_news(ticker)
        all_news.extend(news)

        # Show progress
        print(f"  📊 Current total: {len(all_news)} news items\n")

        # Random delay (important! Avoid being blocked)
        if idx < total:
            delay = random.uniform(3, 7)
            print(f"  ⏳ Waiting {delay:.1f} seconds...\n")
            time.sleep(delay)

    return all_news

def save_to_csv(news_list, output_file):
    """
    Save data to CSV.
    """
    if not news_list:
        print("\n❌ No data to save")
        return None

    # Convert to DataFrame
    df = pd.DataFrame(news_list)

    # Remove duplicates (by ticker and title)
    original_count = len(df)
    df = df.drop_duplicates(subset=['ticker', 'title'], keep='first')
    removed_count = original_count - len(df)

    # Sort by date
    df = df.sort_values(['ticker', 'date'], ascending=[True, False])

    # Save
    df.to_csv(output_file, index=False, encoding='utf-8-sig')

    # Statistics
    print(f"\n{'='*60}")
    print(f"✅ Data saved to: {output_file}")
    print(f"{'='*60}")
    print(f"📊 Data statistics:")
    print(f"  - Original: {original_count} items")
    print(f"  - After deduplication: {len(df)} items")
    print(f"  - Duplicates removed: {removed_count} items")

    print(f"\nNews count per stock:")
    print(df['ticker'].value_counts().sort_index().to_string())

    print(f"\nSource distribution:")
    if 'source' in df.columns:
        print(df['source'].value_counts().head(10).to_string())

    print(f"\nData preview (first 5 items):")
    print(df[['ticker', 'date', 'title']].head().to_string())

    return df

def main():
    """
    Main function
    """
    print("="*60)
    print("📰 Finviz News Scraper")
    print("="*60)
    print(f"Target stocks: {', '.join(TICKERS)}")
    print(f"Output file: {OUTPUT_FILE}")
    print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("="*60)

    # Start fetching
    start_time = time.time()
    news_list = fetch_all_tickers(TICKERS)

    # Save data
    df = save_to_csv(news_list, OUTPUT_FILE)

    # Done
    elapsed = time.time() - start_time
    print(f"\n{'='*60}")
    print(f"✅ Fetch complete!")
    print(f"{'='*60}")
    print(f"⏱️  Total time: {elapsed:.1f} seconds")
    print(f"📁 File location: {OUTPUT_FILE}")

    if df is not None and len(df) < 1000:
        print(f"\n⚠️  Note: Only {len(df)} items currently")
        print(f"Suggestion: Run other scraper scripts to supplement data (Yahoo Finance, MarketWatch)")

if __name__ == "__main__":
    main()

📰 Finviz 新闻爬虫
目标股票: AAPL, MSFT, GOOGL, NVDA, TSLA, JPM, BAC, GS, WMT, DIS
保存文件: finviz_news_data.csv
开始时间: 2025-12-13 20:58:56

开始抓取 10 只股票的新闻

[1/10] 正在处理:  AAPL
----------------------------------------
  📡 正在访问:  https://finviz.com/quote.ashx?t=AAPL
  ✅ AAPL:  成功抓取 100 条新闻
  📊 当前总计: 100 条新闻

  ⏳ 等待 3.5 秒.. .

[2/10] 正在处理:  MSFT
----------------------------------------
  📡 正在访问:  https://finviz.com/quote.ashx?t=MSFT
  ✅ MSFT:  成功抓取 100 条新闻
  📊 当前总计: 200 条新闻

  ⏳ 等待 6.0 秒.. .

[3/10] 正在处理:  GOOGL
----------------------------------------
  📡 正在访问:  https://finviz.com/quote.ashx?t=GOOGL
  ✅ GOOGL:  成功抓取 100 条新闻
  📊 当前总计: 300 条新闻

  ⏳ 等待 5.1 秒.. .

[4/10] 正在处理:  NVDA
----------------------------------------
  📡 正在访问:  https://finviz.com/quote.ashx?t=NVDA
  ✅ NVDA:  成功抓取 100 条新闻
  📊 当前总计: 400 条新闻

  ⏳ 等待 3.3 秒.. .

[5/10] 正在处理:  TSLA
----------------------------------------
  📡 正在访问:  https://finviz.com/quote.ashx?t=TSLA
  ✅ TSLA:  成功抓取 100 条新闻
  📊 当前总计: 500 条新闻

  ⏳ 等待 6.5 秒.. .

[6/10] 正

KeyboardInterrupt: 

##### 1.3: Fetch US stock news from MarketWatch

In [ ]:
# step2d_scrape_marketwatch_simple.py

"""
Simplified MarketWatch scraper (only uses MarketWatch, removes Google).
Focuses on stability and avoids formatting errors.
"""

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from datetime import datetime
import re

# Configuration
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'NVDA', 'TSLA', 'JPM', 'BAC', 'GS', 'WMT', 'DIS']
OUTPUT_FILE = 'marketwatch_news_data.csv'

# Request headers
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
}

def fetch_marketwatch_news(ticker):
    """
    Fetch news from MarketWatch
    """
    news_list = []

    try:
        base_url = "https://www.marketwatch.com/investing/stock/" + ticker.lower()

        print("  Visiting:", base_url)
        response = requests.get(base_url, headers=HEADERS, timeout=15)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            # Find news articles
            articles = soup.find_all('div', class_=re.compile('article__content|element--article'))

            if not articles:
                articles = soup.find_all('article')

            if not articles:
                news_section = soup.find('div', class_=re.compile('latest|news'))
                if news_section:
                    articles = news_section.find_all('a', href=re.compile('/story/'))

            for article in articles[:100]:
                try:
                    title_elem = article.find('h3') or article.find('h2') or article
                    if title_elem:
                        title = title_elem.get_text(strip=True)

                        link_elem = article.find('a') or title_elem.find('a')
                        if link_elem and link_elem.get('href'):
                            url = link_elem['href']
                            if url.startswith('/'):
                                url = "https://www.marketwatch.com" + url

                            # Try to get date
                            time_elem = article.find('time') or article.find('span', class_=re.compile('time|date'))
                            if time_elem:
                                date_str = time_elem.get('datetime', '') or time_elem.get_text(strip=True)
                                try:
                                    if 'T' in date_str:
                                        date_obj = datetime.fromisoformat(date_str. replace('Z', '+00:00'))
                                        formatted_date = date_obj.strftime('%Y-%m-%d')
                                    else:
                                        formatted_date = datetime.now().strftime('%Y-%m-%d')
                                except:
                                    formatted_date = datetime.now().strftime('%Y-%m-%d')
                            else:
                                formatted_date = datetime.now().strftime('%Y-%m-%d')

                            news_list.append({
                                'ticker': ticker,
                                'date': formatted_date,
                                'time': '',
                                'title': title,
                                'url': url,
                                'source': 'MarketWatch',
                                'platform': 'marketwatch'
                            })
                except:
                    continue

            print("  Success:  " + ticker + " fetched " + str(len(news_list)) + " news items")
        else:
            print("  Failed: " + ticker + " HTTP " + str(response.status_code))

    except Exception as e:
        print("  Error:  " + ticker + " - " + str(e))

    return news_list

def main():
    """
    Main function
    """
    print("=" * 60)
    print("MarketWatch News Scraper (Simplified)")
    print("=" * 60)
    print("Target stocks:", ", ".join(TICKERS))
    print("Output file:", OUTPUT_FILE)
    print("=" * 60)
    print()

    all_news = []
    total = len(TICKERS)

    for idx, ticker in enumerate(TICKERS, 1):
        print()
        print("[" + str(idx) + "/" + str(total) + "] Fetching:", ticker)
        print("-" * 50)

        news = fetch_marketwatch_news(ticker)
        all_news.extend(news)

        print("  Current total:", len(all_news), "news items")

        # Delay (use string concatenation to avoid formatting issues)
        if idx < total:
            delay = random. uniform(5, 10)
            delay_str = str(round(delay, 1))
            print("  Waiting " + delay_str + " seconds...")
            print()
            time.sleep(delay)

    # Save data
    if all_news:
        df = pd.DataFrame(all_news)

        # Remove duplicates
        original_count = len(df)
        df = df.drop_duplicates(subset=['ticker', 'title'], keep='first')

        # Save
        df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

        print()
        print("=" * 60)
        print("Data saved:", OUTPUT_FILE)
        print("=" * 60)
        print("Original data:", original_count, "items")
        print("After deduplication:", len(df), "items")
        print()
        print("News count per stock:")
        print(df['ticker'].value_counts().sort_index())
        print()
        print("=" * 60)
        print("Data summary:")
        print("=" * 60)
        print("  Finviz:         996 items")
        print("  MarketWatch:  ", len(df), "items")
        print("  Total:         ", 996 + len(df), "items")
        print("=" * 60)

        if 996 + len(df) >= 1500:
            print()
            print("Congratulations! Enough data, you can proceed to the next step!")
    else:
        print()
        print("No data fetched")

if __name__ == "__main__": 
    main()

MarketWatch 新闻爬虫（简化版）
目标股票: AAPL, MSFT, GOOGL, NVDA, TSLA, JPM, BAC, GS, WMT, DIS
保存文件: marketwatch_news_data.csv


[1/10] 正在抓取: AAPL
--------------------------------------------------
  访问: https://www.marketwatch.com/investing/stock/aapl
  成功:  AAPL 获取 92 条新闻
  当前总计: 92 条新闻
  等待 7.1 秒...


[2/10] 正在抓取: MSFT
--------------------------------------------------
  访问: https://www.marketwatch.com/investing/stock/msft
  成功:  MSFT 获取 92 条新闻
  当前总计: 184 条新闻
  等待 7.2 秒...


[3/10] 正在抓取: GOOGL
--------------------------------------------------
  访问: https://www.marketwatch.com/investing/stock/googl
  成功:  GOOGL 获取 92 条新闻
  当前总计: 276 条新闻
  等待 5.1 秒...


[4/10] 正在抓取: NVDA
--------------------------------------------------
  访问: https://www.marketwatch.com/investing/stock/nvda
  成功:  NVDA 获取 92 条新闻
  当前总计: 368 条新闻
  等待 8.1 秒...


[5/10] 正在抓取: TSLA
--------------------------------------------------
  访问: https://www.marketwatch.com/investing/stock/tsla
  成功:  TSLA 获取 92 条新闻
  当前总计: 460 条新闻
  等待 6.3

##### 1.4: Fetch stock news via Alpha Vantage API

In [ ]:
# step2_alphavantage_news.py

"""
Fetch stock news using Alpha Vantage API.
Official API, stable and reliable.
"""

import requests
import pandas as pd
import time
from datetime import datetime, timedelta

# Configuration
API_KEY = 'R6F8B9TXFSLDCL3Z'  # Your Alpha Vantage API Key
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'NVDA', 'TSLA', 'JPM', 'BAC', 'GS', 'WMT', 'DIS']
OUTPUT_FILE = 'alphavantage_news.csv'

# Alpha Vantage free tier limits
# 5 requests per minute, 500 requests per day

def check_api_key():
    """Check if API Key is valid"""
    print("=" * 60)
    print("Alpha Vantage News API")
    print("=" * 60)
    print()

    if API_KEY == 'YOUR_API_KEY_HERE':
        print("❌ Please set your API Key first!")
        print()
        print("Steps:")
        print("1. If you already have an API Key, enter it at line 12 of the script")
        print("2. If not, visit:  https://www.alphavantage.co/support/#api-key")
        print("   (Free, no credit card required)")
        print()
        return False

    print("✅ API Key is set")
    return True

def fetch_news_alphavantage(ticker, api_key, time_from=None, time_to=None):
    """
    Fetch news from Alpha Vantage.
    
    API docs:   https://www.alphavantage.co/documentation/#news-sentiment
    """
    news_list = []

    try: 
        # Alpha Vantage News & Sentiment API
        url = "https://www.alphavantage.co/query"

        params = {
            'function': 'NEWS_SENTIMENT',
            'tickers': ticker,
            'apikey': api_key,
            'limit': 1000,  # Up to 1000 items (actual may be less)
        }

        # If time range specified
        if time_from: 
            params['time_from'] = time_from  # Format:  20230101T0000
        if time_to:
            params['time_to'] = time_to

        print(f"  Fetching news for {ticker}...")

        response = requests.get(url, params=params, timeout=30)

        if response.status_code == 200:
            data = response.json()

            # Check for error message
            if 'Error Message' in data:
                print(f"  ❌ API Error: {data['Error Message']}")
                return news_list

            if 'Note' in data: 
                print(f"  ⚠️  API Limit: {data['Note']}")
                print("     (Free tier: max 5 requests per minute)")
                return news_list

            # Extract news
            if 'feed' in data: 
                articles = data['feed']

                for article in articles: 
                    try:
                        # Extract info
                        title = article.get('title', '')
                        url_link = article.get('url', '')
                        time_published = article.get('time_published', '')
                        source = article.get('source', '')

                        # Convert time format
                        # Alpha Vantage format: 20231215T120000
                        if time_published:
                            try:
                                date_obj = datetime.strptime(time_published, '%Y%m%dT%H%M%S')
                                formatted_date = date_obj. strftime('%Y-%m-%d')
                                formatted_time = date_obj.strftime('%H:%M:%S')
                            except:
                                formatted_date = time_published[: 8]  # First 8 chars as date
                                formatted_time = ''
                        else:
                            formatted_date = None
                            formatted_time = ''

                        if title and formatted_date:
                            news_list.append({
                                'ticker': ticker,
                                'date': formatted_date,
                                'time': formatted_time,
                                'title': title,
                                'url': url_link,
                                'source': source,
                                'platform': 'alphavantage'
                            })

                    except Exception as e: 
                        continue

                print(f"  ✅ {ticker}:   {len(news_list)} items")
            else:
                print(f"  ⚠️  {ticker}:  No news data found")

        else:
            print(f"  ❌ HTTP {response.status_code}")

    except Exception as e:
        print(f"  ❌ Error: {e}")

    return news_list

def fetch_all_tickers(tickers, api_key, time_from=None, time_to=None):
    """
    Fetch news for all stocks.
    """
    print()
    print("=" * 60)
    print("Start fetching news...")
    print("=" * 60)
    print(f"Number of stocks: {len(tickers)}")
    if time_from:
        print(f"Time range: {time_from} to {time_to}")
    print("=" * 60)
    print()

    all_news = []

    for idx, ticker in enumerate(tickers, 1):
        print(f"[{idx}/{len(tickers)}] {ticker}")
        print("-" * 50)

        news = fetch_news_alphavantage(ticker, api_key, time_from, time_to)
        all_news.extend(news)

        print(f"  📊 Current total: {len(all_news)} items")
        print()

        # Alpha Vantage free tier: max 5 requests per minute
        # So wait at least 12 seconds after each request
        if idx < len(tickers):
            delay = 15  # Wait 15 seconds, safe
            print(f"  ⏳ Waiting {delay} seconds (API limit)...")
            time.sleep(delay)
            print()

    return all_news

def main():
    """Main function"""
    print()
    print("📌 Alpha Vantage News API")
    print()
    print("Advantages:")
    print("  ✅ Official API, stable and reliable")
    print("  ✅ No need for web scraping")
    print("  ✅ High data quality")
    print()
    print("Limits:")
    print("  ⚠️  Free tier: 5 requests per minute, 500 per day")
    print("  ⚠️  10 stocks take about 3-4 minutes")
    print()

    # Check API Key
    if not check_api_key():
        return

    print()
    print("=" * 60)
    print("Time range settings")
    print("=" * 60)
    print()
    print("Options:")
    print("  1. Fetch all available news (no time limit)")
    print("  2. Specify time range (recommended: 2023-2024)")
    print()

    choice = input("Please choose (1 or 2): ").strip()

    time_from = None
    time_to = None

    if choice == '2':
        # Specify time range
        # Alpha Vantage format:   YYYYMMDDTHHMM
        time_from = '20230101T0000'  # Jan 1, 2023
        time_to = '20241231T2359'    # Dec 31, 2024
        print(f"✅ Will fetch news from 2023-01-01 to 2024-12-31")
    else:
        print("✅ Will fetch all available news")

    print()
    input("Press Enter to start...")

    # Start fetching
    all_news = fetch_all_tickers(TICKERS, API_KEY, time_from, time_to)

    # Save data
    if all_news:
        df = pd.DataFrame(all_news)

        # Remove duplicates
        original_count = len(df)
        df = df.drop_duplicates(subset=['ticker', 'title'], keep='first')

        # Save
        df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

        print()
        print("=" * 60)
        print("✅ Done!")
        print("=" * 60)
        print(f"Original data:   {original_count} items")
        print(f"After deduplication: {len(df)} items")
        print(f"Saved to: {OUTPUT_FILE}")
        print()
        print("Stock distribution:")
        print(df['ticker'].value_counts().sort_index())
        print()
        print("Date range:")
        df['date'] = pd.to_datetime(df['date'])
        print(f"  {df['date'].min()} to {df['date'].max()}")

        # Stats for 2023-2024
        df_2023_2024 = df[(df['date'] >= '2023-01-01') & (df['date'] <= '2024-12-31')]
        print()
        print(f"News in 2023-2024:  {len(df_2023_2024)} items")

        if len(df_2023_2024) < 500:
            print()
            print("💡 Note:")
            print("  If there is little historical data, Alpha Vantage mainly provides recent news")
            print("  But the data quality is high, suitable for training")

    else:
        print()
        print("❌ No data fetched")
        print()
        print("Possible reasons:")
        print("  1. Invalid API Key")
        print("  2. Daily request limit reached")
        print("  3. Network issues")

if __name__ == "__main__":
    main()


📌 Alpha Vantage 新闻 API

优势：
  ✅ 官方 API，稳定可靠
  ✅ 不需要爬虫
  ✅ 数据质量高

限制：
  ⚠️  免费版:  每分钟 5 次请求，每天 500 次
  ⚠️  10 只股票需要约 3-4 分钟

Alpha Vantage 新闻 API

✅ API Key 已设置

时间范围设置

选项：
  1. 获取所有可用的新闻（不限时间）
  2. 指定时间范围（推荐：2023-2024年）

✅ 将获取所有可用的新闻


开始获取新闻...
股票数量: 10

[1/10] AAPL
--------------------------------------------------
  正在获取 AAPL 的新闻...
  ✅ AAPL:   1000 条
  📊 当前总计: 1000 条

  ⏳ 等待 15 秒（API 限制）...

[2/10] MSFT
--------------------------------------------------
  正在获取 MSFT 的新闻...
  ✅ MSFT:   1000 条
  📊 当前总计: 2000 条

  ⏳ 等待 15 秒（API 限制）...

[3/10] GOOGL
--------------------------------------------------
  正在获取 GOOGL 的新闻...
  ✅ GOOGL:   1000 条
  📊 当前总计: 3000 条

  ⏳ 等待 15 秒（API 限制）...

[4/10] NVDA
--------------------------------------------------
  正在获取 NVDA 的新闻...
  ✅ NVDA:   1000 条
  📊 当前总计: 4000 条

  ⏳ 等待 15 秒（API 限制）...

[5/10] TSLA
--------------------------------------------------
  正在获取 TSLA 的新闻...
  ✅ TSLA:   1000 条
  📊 当前总计: 5000 条

  ⏳ 等待 15 秒（API 限制）...

[6/10] JPM
-------------

##### 1.5: Merge the three datasets and data cleaning

In [ ]:
# step3a_merge_all_news.py

"""
Step 3A (Updated): Merge all news data.
Includes: Finviz + MarketWatch + Alpha Vantage
"""

import pandas as pd
import re
import os

# Configuration
FINVIZ_FILE = 'finviz_news_data.csv'
MARKETWATCH_FILE = 'marketwatch_news_data.csv'
ALPHAVANTAGE_FILE = 'alphavantage_news.csv'
OUTPUT_FILE = 'combined_clean_news_all.csv'

def clean_text(text):
    """Clean text"""
    if pd.isna(text):
        return ""
    text = str(text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s\.,!?;:\-\'\"\(\)%$]', '', text)
    return text.strip()

def load_and_merge_news():
    """Load and merge all news data"""
    print("=" * 60)
    print("Merge all news data")
    print("=" * 60)
    print()

    all_dataframes = []

    # 1. Load Finviz
    if os.path.exists(FINVIZ_FILE):
        print("Loading Finviz data...")
        try:
            df_finviz = pd.read_csv(FINVIZ_FILE, encoding='utf-8-sig')
            print(f"✅ Finviz:  {len(df_finviz)} items")
            all_dataframes.append(df_finviz)
        except Exception as e:
            print(f"❌ Finviz load failed: {e}")
    else:
        print("⚠️  Not found:", FINVIZ_FILE)

    # 2. Load MarketWatch
    if os.path.exists(MARKETWATCH_FILE):
        print("Loading MarketWatch data...")
        try:
            df_marketwatch = pd.read_csv(MARKETWATCH_FILE, encoding='utf-8-sig')
            print(f"✅ MarketWatch:  {len(df_marketwatch)} items")
            all_dataframes.append(df_marketwatch)
        except Exception as e: 
            print(f"❌ MarketWatch load failed: {e}")
    else:
        print("⚠️  Not found:", MARKETWATCH_FILE)

    # 3. Load Alpha Vantage
    if os.path.exists(ALPHAVANTAGE_FILE):
        print("Loading Alpha Vantage data...")
        try:
            df_alpha = pd.read_csv(ALPHAVANTAGE_FILE, encoding='utf-8-sig')
            print(f"✅ Alpha Vantage: {len(df_alpha)} items")
            all_dataframes.append(df_alpha)
        except Exception as e:
            print(f"❌ Alpha Vantage load failed:  {e}")
    else:
        print("⚠️  Not found:", ALPHAVANTAGE_FILE)

    if len(all_dataframes) == 0:
        print()
        print("❌ No data files available")
        return None

    # Merge all data
    print()
    print("=" * 60)
    print("Merging data...")
    print("=" * 60)

    df_combined = pd.concat(all_dataframes, ignore_index=True)
    print(f"After merge: {len(df_combined)} items")

    return df_combined

def clean_news_data(df):
    """Clean news data"""
    print()
    print("=" * 60)
    print("Cleaning data...")
    print("=" * 60)
    print()

    original_count = len(df)
    print(f"Original data:  {original_count} items")

    # 1. Remove empty titles
    df = df[df['title'].notna()]
    df = df[df['title'].str.strip() != '']
    print(f"After removing empty titles: {len(df)} items")

    # 2. Clean titles
    df['title_clean'] = df['title'].apply(clean_text)

    # 3. Remove short titles
    df = df[df['title_clean'].str.len() >= 10]
    print(f"After removing short titles: {len(df)} items")

    # 4. Remove duplicates (by ticker + title)
    before_dedup = len(df)
    df = df.drop_duplicates(subset=['ticker', 'title_clean'], keep='first')
    print(f"After deduplication:  {len(df)} items (removed {before_dedup - len(df)} duplicates)")

    # 5. Process dates
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df = df[df['date'].notna()]
        print(f"After date processing: {len(df)} items")

        # Sort by date
        df = df.sort_values(['ticker', 'date'], ascending=[True, False])

    # 6. Reset index
    df = df.reset_index(drop=True)

    return df

def show_statistics(df):
    """Show statistics"""
    print()
    print("=" * 60)
    print("Data statistics")
    print("=" * 60)
    print()

    print(f"Final data count: {len(df)} items")
    print()

    # Date range
    if 'date' in df.columns:
        print("Date range:")
        print(f"  Earliest: {df['date'].min()}")
        print(f"  Latest: {df['date'].max()}")
        date_span = (df['date'].max() - df['date'].min()).days
        print(f"  Span: {date_span} days")
        print()

    # Data source distribution
    if 'platform' in df.columns:
        print("Data source distribution:")
        platform_counts = df['platform'].value_counts()
        for platform, count in platform_counts.items():
            percentage = count / len(df) * 100
            print(f"  {platform}: {count} items ({round(percentage, 1)}%)")
        print()

    # Distribution by stock
    print("News count per stock:")
    ticker_counts = df['ticker'].value_counts().sort_index()
    for ticker, count in ticker_counts.items():
        print(f"  {ticker}: {count} items")
    print()

    # Data preview
    print("Data preview (first 5 items):")
    print("-" * 60)
    for i in range(min(5, len(df))):
        row = df.iloc[i]
        date_str = str(row['date'])[:10] if 'date' in df.columns else 'N/A'
        title_preview = row['title_clean'][:60] if 'title_clean' in df.columns else row['title'][:60]
        print(f"{i+1}. {row['ticker']} | {date_str}")
        print(f"   {title_preview}...")
        print()

def save_data(df, output_file):
    """Save data"""
    print("=" * 60)
    print("Saving data...")
    print("=" * 60)

    try:
        df.to_csv(output_file, index=False, encoding='utf-8-sig')

        # Calculate file size
        file_size = os.path.getsize(output_file) / 1024

        print(f"✅ Data saved to:  {output_file}")
        print(f"   Total items: {len(df)}")
        print(f"   File size: {round(file_size, 2)} KB")

    except Exception as e:
        print(f"❌ Save failed: {e}")

def main():
    """Main function"""
    print()
    print("📰 Merge all news data sources")
    print()
    print("Will merge:")
    print("  • Finviz")
    print("  • MarketWatch")
    print("  • Alpha Vantage")
    print()

    # 1. Load and merge
    df_combined = load_and_merge_news()

    if df_combined is None: 
        return

    # 2. Clean data
    df_clean = clean_news_data(df_combined)

    # 3. Show statistics
    show_statistics(df_clean)

    # 4. Save data
    save_data(df_clean, OUTPUT_FILE)

    # 5. Completion message
    print()
    print("=" * 60)
    print("✅ Step 3A complete!")
    print("=" * 60)
    print()
    print(f"Generated file: {OUTPUT_FILE}")
    print(f"Data count: {len(df_clean)}")
    print()
    print("Next step:  Run step3b_match_prices_2025.py")
    print("           Match prices and generate training data")
    print()

if __name__ == "__main__":
    main()


📰 合并所有新闻数据源

将合并:
  • Finviz
  • MarketWatch
  • Alpha Vantage

合并所有新闻数据

加载 Finviz 数据...
✅ Finviz:  996 条
加载 MarketWatch 数据...
✅ MarketWatch:  410 条
加载 Alpha Vantage 数据...
✅ Alpha Vantage: 9390 条

合并数据...
合并后: 10796 条

清洗数据...

原始数据:  10796 条
移除空标题: 10786 条
移除过短标题: 10783 条
去重:  10598 条 (删除了 185 条重复)
处理日期: 10598 条

数据统计

最终数据量: 10598 条

日期范围:
  最早: 2024-02-29 00:00:00
  最晚: 2025-12-13 00:00:00
  跨度: 653 天

数据来源分布:
  alphavantage: 9294 条 (87.7%)
  finviz: 996 条 (9.4%)
  marketwatch: 308 条 (2.9%)

各股票新闻数量:
  AAPL: 1060 条
  BAC: 1053 条
  DIS: 1099 条
  GOOGL: 1071 条
  GS: 1015 条
  JPM: 1065 条
  MSFT: 1071 条
  NVDA: 1091 条
  TSLA: 1003 条
  WMT: 1070 条

数据预览（前5条）:
------------------------------------------------------------
1. AAPL | 2025-12-13
   These 3 Warren Buffett AI Stocks Could Be Big Winners in 202...

2. AAPL | 2025-12-13
   Here's My Top "Magnificent Seven" Stock to Buy for 2026...

3. AAPL | 2025-12-13
   My Tune Is Changing On Apple In 2026: Heres Why...

4. AAPL | 2025-12-13
 

##### 1.6: Generate training set (matching Nov-Dec 2025 news and price data)

In [ ]:
# step3b_match_prices_2025.py
"""
Match Nov-Dec 2025 news and price data.
Uses the newly merged combined_clean_news_all.csv
"""

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

# Configuration
NEWS_FILE = 'combined_clean_news_all.csv'
PRICES_FILE = 'all_stocks_prices.csv'
OUTPUT_FILE = 'training_data_2025.csv'

# Sentiment label thresholds
POSITIVE_THRESHOLD = 1.0   # Gain > 1%
NEGATIVE_THRESHOLD = -1.0  # Loss < -1%

def load_data():
    """Load data"""
    print("=" * 60)
    print("Step 3B: Match 2025 news and prices")
    print("=" * 60)
    print()
    
    print("Loading news data...")
    try:
        df_news = pd.read_csv(NEWS_FILE, encoding='utf-8-sig')
        df_news['date'] = pd.to_datetime(df_news['date'])
        print(f"✅ News data:  {len(df_news)} items")
        print(f"   Date range: {df_news['date'].min()} to {df_news['date'].max()}")
    except Exception as e:
        print(f"❌ Failed to load news: {e}")
        return None, None
    
    print()
    print("Loading price data...")
    try:
        df_prices = pd.read_csv(PRICES_FILE, encoding='utf-8-sig')
        df_prices['date'] = pd.to_datetime(df_prices['date'])
        print(f"✅ Price data: {len(df_prices)} items")
        print(f"   Date range: {df_prices['date'].min()} to {df_prices['date'].max()}")
    except Exception as e: 
        print(f"❌ Failed to load prices: {e}")
        return df_news, None
    
    # Filter Nov-Dec 2025 price data
    print()
    print("Filtering Nov-Dec 2025 price data...")
    df_prices_2025 = df_prices[df_prices['date'] >= '2025-11-01']. copy()
    print(f"✅ Nov-Dec 2025 prices:  {len(df_prices_2025)} items")
    
    if len(df_prices_2025) == 0:
        print()
        print("⚠️  Warning:  No Nov-Dec 2025 price data!")
        print("   You may need to update price data to latest")
        return df_news, None
    
    return df_news, df_prices_2025

def calculate_price_change(df_prices, ticker, news_date):
    """
    Calculate price change.
    
    Strategy:
    1. Find the closing price on or before the news date (as baseline)
    2. Find the closing price 1-5 days after the news date
    3. Calculate percentage change
    """
    try:
        # Filter data for this stock
        stock_data = df_prices[df_prices['ticker'] == ticker].copy()
        
        if len(stock_data) == 0:
            return None
        
        stock_data = stock_data.sort_values('date')
        
        # Method 1: Find the nearest price on or before news date
        before_data = stock_data[stock_data['date'] <= news_date]
        
        if len(before_data) == 0:
            # If news date is too early, try to find earliest price
            before_data = stock_data
        
        if len(before_data) == 0:
            return None
        
        price_before = before_data.iloc[-1]['close']
        date_before = before_data.iloc[-1]['date']
        
        # Method 2: Find price 1-5 days after news date
        after_data = stock_data[
            (stock_data['date'] > news_date) & 
            (stock_data['date'] <= news_date + timedelta(days=5))
        ]
        
        if len(after_data) == 0:
            # If no data after, try same day data
            same_day = stock_data[stock_data['date'] == news_date]
            
            if len(same_day) > 0:
                # Use same day close vs open
                row = same_day.iloc[0]
                price_before = row['open']
                price_after = row['close']
            else:
                # No matching data at all
                return None
        else:
            price_after = after_data.iloc[0]['close']
            date_after = after_data.iloc[0]['date']
        
        # Calculate percentage change
        if price_before > 0:
            price_change = ((price_after - price_before) / price_before) * 100
            return round(price_change, 4)
        else:
            return None
    
    except Exception as e: 
        return None

def assign_sentiment(price_change):
    """
    Assign sentiment label based on price change.
    
    positive: Gain > 1%
    negative: Loss < -1%
    neutral:  -1% ~ 1%
    """
    if price_change is None:
        return 'unknown'
    elif price_change > POSITIVE_THRESHOLD:
        return 'positive'
    elif price_change < NEGATIVE_THRESHOLD:
        return 'negative'
    else:
        return 'neutral'

def match_and_label(df_news, df_prices):
    """Match prices and label sentiment"""
    print()
    print("=" * 60)
    print("Starting price matching and labeling...")
    print("=" * 60)
    print()
    
    total = len(df_news)
    print(f"Total to process:  {total} news items")
    print()
    
    price_changes = []
    sentiments = []
    matched_count = 0
    
    start_time = time.time()
    
    for idx, row in df_news.iterrows():
        # Show progress
        if (idx + 1) % 500 == 0 or (idx + 1) == total:
            progress = (idx + 1) / total * 100
            elapsed = time.time() - start_time
            speed = (idx + 1) / elapsed
            remaining = (total - idx - 1) / speed if speed > 0 else 0
            
            print(f"Progress: {idx + 1}/{total} ({progress:.1f}%) | "
                  f"Matched: {matched_count} | "
                  f"Speed:  {speed:.1f} items/sec | "
                  f"Remaining: {int(remaining)} sec")
        
        # Calculate price change
        price_change = calculate_price_change(
            df_prices,
            row['ticker'],
            row['date']
        )
        
        if price_change is not None:
            matched_count += 1
        
        price_changes.append(price_change)
        
        # Assign sentiment label
        sentiment = assign_sentiment(price_change)
        sentiments.append(sentiment)
    
    # Add to DataFrame
    df_news['price_change'] = price_changes
    df_news['sentiment'] = sentiments
    
    print()
    print("=" * 60)
    print("Matching complete!")
    print("=" * 60)
    print(f"Total data:  {total} items")
    print(f"Successfully matched: {matched_count} items")
    print(f"Match rate: {matched_count/total*100:.1f}%")
    print(f"Total time: {time.time() - start_time:.1f} sec")
    
    return df_news

def show_statistics(df):
    """Show statistics"""
    print()
    print("=" * 60)
    print("Data statistics")
    print("=" * 60)
    print()
    
    # Remove unknown
    df_valid = df[df['sentiment'] != 'unknown'].copy()
    
    print(f"Total data: {len(df)} items")
    print(f"Valid data: {len(df_valid)} items")
    print(f"Invalid data: {len(df) - len(df_valid)} items")
    
    if len(df_valid) == 0:
        print()
        print("❌ No valid data!")
        return None
    
    print()
    print("Sentiment label distribution:")
    print("-" * 60)
    sentiment_counts = df_valid['sentiment'].value_counts()
    for sentiment in ['positive', 'negative', 'neutral']:
        if sentiment in sentiment_counts.index:
            count = sentiment_counts[sentiment]
            percentage = count / len(df_valid) * 100
            bar = "█" * int(percentage / 2)
            print(f"  {sentiment.ljust(10)}: {str(count).rjust(5)} items ({percentage:5.1f}%) {bar}")
    
    print()
    print("Price change statistics:")
    print(f"  Mean change: {df_valid['price_change'].mean():.2f}%")
    print(f"  Std dev: {df_valid['price_change'].std():.2f}%")
    print(f"  Max gain: {df_valid['price_change'].max():.2f}%")
    print(f"  Max loss: {df_valid['price_change'].min():.2f}%")
    
    print()
    print("Data distribution by stock:")
    print("-" * 60)
    print(f"{'Ticker':<8} {'Positive':>10} {'Negative':>10} {'Neutral':>10} {'Total':>10}")
    print("-" * 60)
    
    for ticker in sorted(df_valid['ticker'].unique()):
        ticker_data = df_valid[df_valid['ticker'] == ticker]
        pos = len(ticker_data[ticker_data['sentiment'] == 'positive'])
        neg = len(ticker_data[ticker_data['sentiment'] == 'negative'])
        neu = len(ticker_data[ticker_data['sentiment'] == 'neutral'])
        total_ticker = len(ticker_data)
        
        print(f"{ticker:<8} {pos:>10} {neg:>10} {neu:>10} {total_ticker:>10}")
    
    return df_valid

def prepare_training_data(df):
    """Prepare training data"""
    print()
    print("=" * 60)
    print("Preparing training data...")
    print("=" * 60)
    print()
    
    # Select required columns
    if 'title_clean' in df.columns:
        text_col = 'title_clean'
    elif 'title' in df.columns:
        text_col = 'title'
    else:
        print("❌ Title column not found")
        return None
    
    training_df = df[[
        'ticker',
        'date',
        text_col,
        'sentiment',
        'price_change'
    ]].copy()
    
    # Rename columns
    training_df = training_df.rename(columns={
        text_col: 'text',
        'sentiment': 'label'
    })
    
    # Ensure text is not empty
    training_df = training_df[training_df['text'].notna()]
    training_df = training_df[training_df['text'].str.strip() != '']
    
    print(f"✅ Training data ready: {len(training_df)} items")
    
    return training_df

def show_samples(df):
    """Show sample data"""
    print()
    print("=" * 60)
    print("Data samples (3 per category)")
    print("=" * 60)
    print()
    
    for label in ['positive', 'negative', 'neutral']:
        label_data = df[df['label'] == label]
        
        if len(label_data) > 0:
            print(f"{label.upper()}:")
            print("-" * 60)
            
            samples = label_data.sample(min(3, len(label_data)))
            
            for idx, (i, row) in enumerate(samples.iterrows(), 1):
                date_str = str(row['date'])[:10]
                change = row['price_change']
                ticker = row['ticker']
                text = row['text'][:80]
                
                print(f"{idx}. {ticker} | {date_str} | Change: {change:+.2f}%")
                print(f"   {text}")
                if len(row['text']) > 80:
                    print("   ...")
                print()

def save_data(df, output_file):
    """Save data"""
    print()
    print("=" * 60)
    print("Saving data...")
    print("=" * 60)
    
    try:
        df.to_csv(output_file, index=False, encoding='utf-8-sig')
        
        import os
        file_size = os.path.getsize(output_file) / 1024
        
        print(f"✅ Data saved!")
        print(f"   File:  {output_file}")
        print(f"   Count: {len(df)}")
        print(f"   Size: {file_size:.2f} KB")
        
    except Exception as e:
        print(f"❌ Save failed: {e}")

def main():
    """Main function"""
    print()
    print("🎯 Match 2025 news and price data")
    print()
    print("Configuration:")
    print(f"  News file: {NEWS_FILE}")
    print(f"  Price file: {PRICES_FILE}")
    print(f"  Output file: {OUTPUT_FILE}")
    print(f"  Thresholds: positive>{POSITIVE_THRESHOLD}%, negative<{NEGATIVE_THRESHOLD}%")
    print()
    
    # 1. Load data
    df_news, df_prices = load_data()
    
    if df_news is None or df_prices is None:
        print()
        print("❌ Data loading failed, cannot continue")
        return
    
    # 2. Match and label
    df_labeled = match_and_label(df_news, df_prices)
    
    # 3. Show statistics
    df_valid = show_statistics(df_labeled)
    
    if df_valid is None or len(df_valid) < 100:
        print()
        print("⚠️  Too few valid data, please check price data")
        return
    
    # 4. Prepare training data
    training_df = prepare_training_data(df_valid)
    
    if training_df is None:
        return
    
    # 5. Show samples
    show_samples(training_df)
    
    # 6. Save data
    save_data(training_df, OUTPUT_FILE)
    
    # 7. Final summary
    print()
    print("=" * 60)
    print("✅ All complete!")
    print("=" * 60)
    print()
    print(f"📊 Final training data:  {len(training_df)} items")
    print()
    print("Label distribution:")
    label_dist = training_df['label'].value_counts()
    for label, count in label_dist.items():
        percentage = count / len(training_df) * 100
        print(f"  {label}: {count} items ({percentage:.1f}%)")
    print()
    print("🎉 Ready to train FinBERT model!")
    print()

if __name__ == "__main__": 
    main()


🎯 匹配2025年新闻和价格数据

配置:
  新闻文件: combined_clean_news_all.csv
  价格文件: all_stocks_prices.csv
  输出文件: training_data_2025.csv
  阈值: positive>1.0%, negative<-1.0%


第三步B：匹配2025年新闻和价格

加载新闻数据...
✅ 新闻数据:  10598 条
   日期范围: 2024-02-29 00:00:00 到 2025-12-13 00:00:00

加载价格数据...
✅ 价格数据: 7400 条
   日期范围: 2023-01-03 00:00:00 到 2025-12-12 00:00:00

筛选2025年11-12月的价格数据...
✅ 2025年11-12月价格:  290 条

开始匹配价格并标注...

总共需要处理:  10598 条新闻

进度: 500/10598 (4.7%) | 已匹配: 367 | 速度:  169.5 条/秒 | 预计剩余: 59 秒
进度: 1000/10598 (9.4%) | 已匹配: 835 | 速度:  172.1 条/秒 | 预计剩余: 55 秒
进度: 1500/10598 (14.2%) | 已匹配: 1150 | 速度:  168.6 条/秒 | 预计剩余: 53 秒
进度: 2000/10598 (18.9%) | 已匹配: 1650 | 速度:  164.1 条/秒 | 预计剩余: 52 秒
进度: 2500/10598 (23.6%) | 已匹配: 2030 | 速度:  165.1 条/秒 | 预计剩余: 49 秒
进度: 3000/10598 (28.3%) | 已匹配: 2060 | 速度:  165.2 条/秒 | 预计剩余: 46 秒
进度: 3500/10598 (33.0%) | 已匹配: 2216 | 速度:  165.4 条/秒 | 预计剩余: 42 秒
进度: 4000/10598 (37.7%) | 已匹配: 2716 | 速度:  166.4 条/秒 | 预计剩余: 39 秒
进度: 4500/10598 (42.5%) | 已匹配: 3104 | 速度:  166.8 条/秒 | 预计剩余: 36 秒
进度: 50

##### 1.7: Data quality analysis

In [ ]:
# Data quality analysis
# analyze_training_data.py
"""
Detailed analysis of training set quality.
Includes: data distribution, text quality, price changes, time distribution, etc.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Configuration
DATA_FILE = 'training_data_2025.csv'

# Set font for display
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

def load_data():
    """Load data"""
    print("=" * 80)
    print("Training Set Quality Analysis Report")
    print("=" * 80)
    print()
    
    print("Loading data...")
    df = pd.read_csv(DATA_FILE, encoding='utf-8-sig')
    df['date'] = pd.to_datetime(df['date'])
    
    print(f"✅ Data loaded successfully:   {len(df)} items")
    print()
    
    return df

def basic_statistics(df):
    """Basic statistics"""
    print("=" * 80)
    print("1. Basic Statistics")
    print("=" * 80)
    print()
    
    print(f"📊 Total data: {len(df)} items")
    print(f"📅 Date range:   {df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")
    print(f"📆 Time span:   {(df['date'].max() - df['date'].min()).days} days")
    print(f"🏢 Stock count: {df['ticker'].nunique()} stocks")
    print()
    
    # Data column info
    print("Data columns:")
    for col in df.columns:
        print(f"  • {col}:  {df[col].dtype}")
    print()

def label_distribution(df):
    """Label distribution analysis"""
    print("=" * 80)
    print("2. Label Distribution")
    print("=" * 80)
    print()
    
    label_counts = df['label'].value_counts()
    
    print("Label statistics:")
    print("-" * 80)
    print(f"{'Label':<12} {'Count':>8} {'Percentage':>10} {'Visualization':>20}")
    print("-" * 80)
    
    for label, count in label_counts.items():
        percentage = count / len(df) * 100
        bar = "█" * int(percentage / 2)
        print(f"{label:<12} {count:>8} {percentage:>9.2f}% {bar}")
    
    print("-" * 80)
    print(f"{'Total':<12} {len(df):>8} {'100.00':>9}%")
    print()
    
    # Balance analysis
    print("Data balance:")
    max_count = label_counts.max()
    min_count = label_counts.min()
    balance_ratio = min_count / max_count
    
    print(f"  Most common: {label_counts.idxmax()} ({max_count} items)")
    print(f"  Least common: {label_counts.idxmin()} ({min_count} items)")
    print(f"  Balance ratio: {balance_ratio:.2f} (min/max)")
    
    if balance_ratio >= 0.8:
        print(f"  Rating:  ⭐⭐⭐⭐⭐ Very balanced")
    elif balance_ratio >= 0.6:
        print(f"  Rating: ⭐⭐⭐⭐ Fairly balanced")
    elif balance_ratio >= 0.4:
        print(f"  Rating: ⭐⭐⭐ Basically balanced")
    else:
        print(f"  Rating: ⭐⭐ Not balanced, adjustment recommended")
    
    print()

def price_change_analysis(df):
    """Price change analysis"""
    print("=" * 80)
    print("3. Price Change Analysis")
    print("=" * 80)
    print()
    
    print("Overall price change statistics:")
    print("-" * 80)
    print(f"  Mean change: {df['price_change'].mean():.4f}%")
    print(f"  Median:   {df['price_change'].median():.4f}%")
    print(f"  Std dev:  {df['price_change'].std():.4f}%")
    print(f"  Max gain: {df['price_change'].max():.4f}%")
    print(f"  Max loss: {df['price_change'].min():.4f}%")
    print()
    
    # Stats by label
    print("Price change by label:")
    print("-" * 80)
    print(f"{'Label':<12} {'Mean':>10} {'Median':>10} {'Std Dev':>10} {'Max':>10} {'Min':>10}")
    print("-" * 80)
    
    for label in ['positive', 'negative', 'neutral']:
        if label in df['label'].values:
            label_data = df[df['label'] == label]['price_change']
            print(f"{label:<12} {label_data.mean():>9.2f}% {label_data.median():>9.2f}% "
                  f"{label_data.std():>9.2f}% {label_data.max():>9.2f}% {label_data.min():>9.2f}%")
    
    print()
    
    # Price change distribution ranges
    print("Price change range distribution:")
    print("-" * 80)
    bins = [
        (-float('inf'), -5, "Big drop (<-5%)"),
        (-5, -2, "Medium drop (-5%~-2%)"),
        (-2, -1, "Small drop (-2%~-1%)"),
        (-1, 1, "Flat (-1%~1%)"),
        (1, 2, "Small gain (1%~2%)"),
        (2, 5, "Medium gain (2%~5%)"),
        (5, float('inf'), "Big gain (>5%)")
    ]
    
    for low, high, desc in bins:
        count = len(df[(df['price_change'] > low) & (df['price_change'] <= high)])
        percentage = count / len(df) * 100
        bar = "█" * int(percentage / 2)
        print(f"  {desc:<20} {count:>5} items ({percentage:>5.1f}%) {bar}")
    
    print()

def ticker_distribution(df):
    """Stock distribution analysis"""
    print("=" * 80)
    print("4. Stock Distribution Analysis")
    print("=" * 80)
    print()
    
    ticker_counts = df['ticker'].value_counts()
    
    print("Data count per stock:")
    print("-" * 80)
    print(f"{'Ticker':<8} {'Total':>8} {'Positive':>10} {'Negative':>10} {'Neutral':>10}")
    print("-" * 80)
    
    for ticker in sorted(ticker_counts.index):
        ticker_data = df[df['ticker'] == ticker]
        total = len(ticker_data)
        pos = len(ticker_data[ticker_data['label'] == 'positive'])
        neg = len(ticker_data[ticker_data['label'] == 'negative'])
        neu = len(ticker_data[ticker_data['label'] == 'neutral'])
        
        print(f"{ticker:<8} {total:>8} {pos:>10} {neg:>10} {neu:>10}")
    
    print("-" * 80)
    print(f"{'Total':<8} {len(df):>8}")
    print()
    
    # Stock distribution balance
    print("Stock distribution balance:")
    max_ticker_count = ticker_counts.max()
    min_ticker_count = ticker_counts.min()
    ticker_balance = min_ticker_count / max_ticker_count
    
    print(f"  Most:  {ticker_counts.idxmax()} ({max_ticker_count} items)")
    print(f"  Least: {ticker_counts.idxmin()} ({min_ticker_count} items)")
    print(f"  Balance ratio: {ticker_balance:.2f}")
    
    if ticker_balance >= 0.7:
        print(f"  Rating: ⭐⭐⭐⭐⭐ Stock distribution is balanced")
    elif ticker_balance >= 0.5:
        print(f"  Rating: ⭐⭐⭐⭐ Stock distribution is fairly balanced")
    else:
        print(f"  Rating: ⭐⭐⭐ Stock distribution has variance")
    
    print()

def text_quality_analysis(df):
    """Text quality analysis"""
    print("=" * 80)
    print("5. Text Quality Analysis")
    print("=" * 80)
    print()
    
    # Text length statistics
    df['text_length'] = df['text'].str.len()
    df['word_count'] = df['text'].str.split().str.len()
    
    print("Text length statistics:")
    print("-" * 80)
    print(f"  Avg chars: {df['text_length'].mean():.1f}")
    print(f"  Median:     {df['text_length'].median():.1f}")
    print(f"  Shortest:       {df['text_length'].min()} chars")
    print(f"  Longest:      {df['text_length'].max()} chars")
    print()
    
    print(f"  Avg words: {df['word_count'].mean():.1f}")
    print(f"  Median:    {df['word_count'].median():.1f}")
    print(f"  Fewest:      {df['word_count'].min()} words")
    print(f"  Most:      {df['word_count'].max()} words")
    print()
    
    # Text length distribution
    print("Text length distribution:")
    print("-" * 80)
    length_bins = [
        (0, 50, "Very short (<50 chars)"),
        (50, 100, "Short (50-100)"),
        (100, 150, "Medium (100-150)"),
        (150, 200, "Long (150-200)"),
        (200, float('inf'), "Very long (>200)")
    ]
    
    for low, high, desc in length_bins:
        count = len(df[(df['text_length'] > low) & (df['text_length'] <= high)])
        percentage = count / len(df) * 100
        bar = "█" * int(percentage / 2)
        print(f"  {desc:<25} {count:>5} items ({percentage:>5.1f}%) {bar}")
    
    print()
    
    # Check nulls and duplicates
    print("Data completeness:")
    print("-" * 80)
    null_count = df['text'].isnull().sum()
    empty_count = (df['text'].str.strip() == '').sum()
    duplicate_count = df.duplicated(subset=['ticker', 'text']).sum()
    
    print(f"  Null count: {null_count}")
    print(f"  Empty text:   {empty_count}")
    print(f"  Duplicates: {duplicate_count}")
    
    if null_count == 0 and empty_count == 0:
        print(f"  Rating: ⭐⭐⭐⭐⭐ Data is complete")
    else:
        print(f"  Rating: ⚠️  Issues exist, cleaning needed")
    
    print()

def time_distribution(df):
    """Time distribution analysis"""
    print("=" * 80)
    print("6. Time Distribution Analysis")
    print("=" * 80)
    print()
    
    # Monthly stats
    df['year_month'] = df['date'].dt.to_period('M')
    monthly_counts = df.groupby('year_month').size().sort_index()
    
    print("Monthly distribution:")
    print("-" * 80)
    for period, count in monthly_counts.items():
        percentage = count / len(df) * 100
        bar = "█" * int(percentage / 2)
        print(f"  {period}:  {count:>5} items ({percentage:>5.1f}%) {bar}")
    
    print()
    
    # Weekday stats
    df['weekday'] = df['date'].dt.day_name()
    weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    weekday_counts = df['weekday'].value_counts()
    
    print("Weekday distribution:")
    print("-" * 80)
    for day in weekday_order:
        if day in weekday_counts.index:
            count = weekday_counts[day]
            percentage = count / len(df) * 100
            bar = "█" * int(percentage / 3)
            print(f"  {day:<10}:  {count:>5} items ({percentage:>5.1f}%) {bar}")
    
    print()

def sample_display(df):
    """Sample display"""
    print("=" * 80)
    print("7. Sample Data Display")
    print("=" * 80)
    print()
    
    for label in ['positive', 'negative', 'neutral']:
        if label in df['label'].values:
            print(f"{label.upper()} samples (random 3):")
            print("-" * 80)
            
            samples = df[df['label'] == label].sample(min(3, len(df[df['label'] == label])))
            
            for idx, (i, row) in enumerate(samples.iterrows(), 1):
                print(f"\n{idx}. Ticker: {row['ticker']} | Date: {row['date'].strftime('%Y-%m-%d')} | "
                      f"Price change: {row['price_change']:+.2f}%")
                print(f"   Text:  {row['text'][:100]}")
                if len(row['text']) > 100:
                    print(f"        {row['text'][100:200]}")
                    if len(row['text']) > 200:
                        print(f"        ...")
            print()

def overall_quality_score(df):
    """Overall quality score"""
    print("=" * 80)
    print("8. Overall Quality Score")
    print("=" * 80)
    print()
    
    scores = {}
    
    # 1. Data size (20 points)
    data_size = len(df)
    if data_size >= 8000:
        scores['Data Size'] = 20
    elif data_size >= 5000:
        scores['Data Size'] = 15
    elif data_size >= 3000:
        scores['Data Size'] = 10
    else:
        scores['Data Size'] = 5
    
    # 2. Label balance (20 points)
    label_counts = df['label'].value_counts()
    balance_ratio = label_counts.min() / label_counts.max()
    if balance_ratio >= 0.8:
        scores['Label Balance'] = 20
    elif balance_ratio >= 0.6:
        scores['Label Balance'] = 15
    elif balance_ratio >= 0.4:
        scores['Label Balance'] = 10
    else:
        scores['Label Balance'] = 5
    
    # 3. Stock coverage (15 points)
    ticker_count = df['ticker'].nunique()
    if ticker_count >= 10:
        scores['Stock Coverage'] = 15
    elif ticker_count >= 7:
        scores['Stock Coverage'] = 10
    else:
        scores['Stock Coverage'] = 5
    
    # 4. Text quality (15 points)
    avg_length = df['text'].str.len().mean()
    null_count = df['text'].isnull().sum()
    if avg_length >= 80 and null_count == 0:
        scores['Text Quality'] = 15
    elif avg_length >= 50 and null_count == 0:
        scores['Text Quality'] = 10
    else:
        scores['Text Quality'] = 5
    
    # 5. Price change reasonability (15 points)
    price_std = df['price_change'].std()
    if 1.0 <= price_std <= 5.0:
        scores['Price Change'] = 15
    elif 0.5 <= price_std <= 8.0:
        scores['Price Change'] = 10
    else:
        scores['Price Change'] = 5
    
    # 6. Data completeness (15 points)
    complete_ratio = 1 - (df.isnull().sum().sum() / (len(df) * len(df.columns)))
    if complete_ratio >= 0.99:
        scores['Completeness'] = 15
    elif complete_ratio >= 0.95:
        scores['Completeness'] = 10
    else:
        scores['Completeness'] = 5
    
    # Display scores
    print("Score breakdown:")
    print("-" * 80)
    print(f"{'Category':<18} {'Score':>8} {'Max':>8} {'Grade':>10}")
    print("-" * 80)
    
    total_score = 0
    for item, score in scores.items():
        if item == 'Data Size' or item == 'Label Balance': 
            max_score = 20
        else:
            max_score = 15
        
        total_score += score
        
        if score >= max_score * 0.9:
            grade = "⭐⭐⭐⭐⭐"
        elif score >= max_score * 0.7:
            grade = "⭐⭐⭐⭐"
        elif score >= max_score * 0.5:
            grade = "⭐⭐⭐"
        else:
            grade = "⭐⭐"
        
        print(f"{item:<18} {score:>8} {max_score:>8} {grade:>10}")
    
    print("-" * 80)
    print(f"{'Total':<18} {total_score:>8} {'100':>8}")
    print()
    
    # Overall assessment
    if total_score >= 90:
        overall = "⭐⭐⭐⭐⭐ Excellent - Very high data quality, ready for training"
    elif total_score >= 75:
        overall = "⭐⭐⭐⭐ Good - High data quality, suitable for training"
    elif total_score >= 60:
        overall = "⭐⭐⭐ Fair - Acceptable quality, optimization recommended"
    else: 
        overall = "⭐⭐ Poor - Data quality improvement needed"
    
    print(f"Overall assessment: {overall}")
    print()

def main():
    """Main function"""
    # Load data
    df = load_data()
    
    # 1. Basic statistics
    basic_statistics(df)
    
    # 2. Label distribution
    label_distribution(df)
    
    # 3. Price change analysis
    price_change_analysis(df)
    
    # 4. Stock distribution
    ticker_distribution(df)
    
    # 5. Text quality
    text_quality_analysis(df)
    
    # 6. Time distribution
    time_distribution(df)
    
    # 7. Sample display
    sample_display(df)
    
    # 8. Overall score
    overall_quality_score(df)
    
    # Complete
    print("=" * 80)
    print("✅ Analysis complete!")
    print("=" * 80)
    print()
    print("Recommendations:")
    print("  1. If overall score >= 75, you can directly use this for FinBERT training")
    print("  2. If any category scores low, you can optimize specifically")
    print("  3. Consider saving the analysis results for your project report")
    print()

if __name__ == "__main__": 
    main()

训练集质量分析报告

加载数据...
✅ 数据加载成功:   8377 条

1. 基础统计

📊 总数据量: 8377 条
📅 日期范围:   2025-10-29 到 2025-12-12
📆 时间跨度:   44 天
🏢 股票数量: 10 只

数据列:
  • ticker:  object
  • date:  datetime64[ns]
  • text:  object
  • label:  object
  • price_change:  float64

2. 标签分布

标签统计:
--------------------------------------------------------------------------------
标签                 数量        百分比                  可视化
--------------------------------------------------------------------------------
neutral          3374     40.28% ████████████████████
positive         3020     36.05% ██████████████████
negative         1983     23.67% ███████████
--------------------------------------------------------------------------------
总计               8377   100. 00%

数据平衡性:
  最多类别: neutral (3374 条)
  最少类别: negative (1983 条)
  平衡比例: 0.59 (最少/最多)
  评估: ⭐⭐⭐ 基本平衡

3. 价格变化分析

整体价格变化统计:
--------------------------------------------------------------------------------
  平均变化: 0.1414%
  中位数:   0.3592%
  标准差:  2.2911%
  最大涨幅: 6.8245%

##### 1.8: Convert training set from 3-class to 7-class (Surge, Rise, Small_rise, Stable, Small_drop, drop, crash)

In [ ]:
# step4_relabel_7class.py
"""
Step 4: Relabel training data from 3-class to 7-class
Divide into 7 sub-intervals based on actual price changes
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Configuration
INPUT_FILE = 'training_data_2025.csv'
OUTPUT_FILE = 'financial_data/training_data_7class.csv'

# 7 class definitions
LABEL_DEFINITIONS = {
    'crash': {
        'name': '大跌',
        'range': '< -5%',
        'threshold': (-float('inf'), -5)
    },
    'drop': {
        'name': '中跌',
        'range': '-5% ~ -2%',
        'threshold': (-5, -2)
    },
    'small_drop': {
        'name': '小跌',
        'range': '-2% ~ -1%',
        'threshold': (-2, -1)
    },
    'stable': {
        'name': '震荡',
        'range': '-1% ~ 1%',
        'threshold': (-1, 1)
    },
    'small_rise': {
        'name': '小涨',
        'range': '1% ~ 2%',
        'threshold': (1, 2)
    },
    'rise': {
        'name': '中涨',
        'range': '2% ~ 5%',
        'threshold': (2, 5)
    },
    'surge': {
        'name': '大涨',
        'range': '> 5%',
        'threshold': (5, float('inf'))
    }
}

# Label order (from big loss to big gain)
LABELS_ORDER = ['crash', 'drop', 'small_drop', 'stable', 'small_rise', 'rise', 'surge']

def print_section(title):
    """Print section divider"""
    print("\n" + "=" * 80)
    print(title)
    print("=" * 80 + "\n")

def assign_7class_label(price_change):
    """
    Assign 7-class label based on price change
    
    Args: 
        price_change: Price change percentage
        
    Returns:
        Label name
    """
    if pd.isna(price_change):
        return None
    
    for label in LABELS_ORDER:
        low, high = LABEL_DEFINITIONS[label]['threshold']
        if low < price_change <= high:
            return label
    
    # Fallback: if no match (theoretically shouldn't happen)
    if price_change <= -5:
        return 'crash'
    else:
        return 'surge'

def load_and_relabel():
    """Load data and relabel"""
    print_section("📂 Step 1: Load original data")
    
    # Read data
    if not os.path.exists(INPUT_FILE):
        print(f"❌ Error: Cannot find file {INPUT_FILE}")
        return None
    
    df = pd.read_csv(INPUT_FILE, encoding='utf-8-sig')
    print(f"✅ Data loaded successfully: {len(df)} entries")
    print()
    
    # Check required columns
    if 'price_change' not in df.columns:
        print("❌ Error: Data does not have 'price_change' column")
        return None
    
    # Show original 3-class distribution
    if 'label' in df.columns:
        print("Original 3-class label distribution:")
        print("-" * 60)
        for label in ['positive', 'negative', 'neutral']:
            if label in df['label'].values:
                count = len(df[df['label'] == label])
                percentage = count / len(df) * 100
                bar = "█" * int(percentage / 2)
                print(f"  {label.ljust(10)}: {count:>5} entries ({percentage:5.1f}%) {bar}")
        print()
    
    # Relabel to 7-class
    print_section("🏷️ Step 2: Relabel to 7-class")
    
    print("Labeling rules:")
    print("-" * 60)
    for label in LABELS_ORDER: 
        info = LABEL_DEFINITIONS[label]
        print(f"  {label.ljust(12)} ({info['name']}): {info['range']}")
    print()
    
    print("Starting labeling...")
    df['label_7class'] = df['price_change'].apply(assign_7class_label)
    
    # Count labeling results
    labeled_count = df['label_7class'].notna().sum()
    print(f"✅ Labeling complete: {labeled_count} / {len(df)} entries")
    
    if labeled_count < len(df):
        null_count = len(df) - labeled_count
        print(f"⚠️ {null_count} entries cannot be labeled (price_change is null)")
        df = df[df['label_7class'].notna()].copy()
        print(f"   Removed, remaining {len(df)} entries")
    
    print()
    
    return df

def analyze_distribution(df):
    """Analyze 7-class label distribution"""
    print_section("📊 Step 3: Analyze new label distribution")
    
    print("7-class label statistics:")
    print("-" * 80)
    print(f"{'Label':<12} {'Name':<12} {'Range':<15} {'Count':>8} {'Percentage':>10} {'Visualization':<30}")
    print("-" * 80)
    
    total = len(df)
    
    for label in LABELS_ORDER:
        info = LABEL_DEFINITIONS[label]
        count = len(df[df['label_7class'] == label])
        percentage = count / total * 100
        bar = "█" * int(percentage / 2)
        
        print(f"{label:<12} {info['name']:<12} {info['range']:<15} {count:>8} {percentage:>9.1f}% {bar}")
    
    print("-" * 80)
    print(f"{'Total':<39} {total:>8} {'100.0':>9}%")
    print()
    
    # Data balance analysis
    print("Data balance analysis:")
    print("-" * 60)
    
    counts = df['label_7class'].value_counts()
    max_count = counts.max()
    min_count = counts.min()
    balance_ratio = min_count / max_count
    
    print(f"  Most frequent class: {counts.idxmax()} ({max_count} entries)")
    print(f"  Least frequent class: {counts.idxmin()} ({min_count} entries)")
    print(f"  Balance ratio: {balance_ratio:.3f} (min/max)")
    print()
    
    if balance_ratio >= 0.3:
        print("  Assessment: ⭐⭐⭐⭐ Distribution is reasonably balanced, suitable for training")
    elif balance_ratio >= 0.1:
        print("  Assessment: ⭐⭐⭐ Distribution is somewhat uneven, but usable")
        print("  Suggestion: Consider using class weights during training")
    else:
        print("  Assessment: ⭐⭐ Distribution is very unbalanced")
        print("  Suggestion: Consider merging similar classes or using oversampling")
    print()
    
    # Price change statistics for each label
    print("Price change statistics for each label:")
    print("-" * 80)
    print(f"{'Label':<12} {'Mean':>10} {'Median':>10} {'Std Dev':>10} {'Min':>10} {'Max':>10}")
    print("-" * 80)
    
    for label in LABELS_ORDER:
        label_data = df[df['label_7class'] == label]['price_change']
        if len(label_data) > 0:
            print(f"{label:<12} {label_data.mean():>9.2f}% {label_data.median():>9.2f}% "
                  f"{label_data.std():>9.2f}% {label_data.min():>9.2f}% {label_data.max():>9.2f}%")
    
    print()

def show_samples(df):
    """Show samples for each category"""
    print_section("🔍 Step 4: View sample data")
    
    for label in LABELS_ORDER: 
        label_data = df[df['label_7class'] == label]
        
        if len(label_data) == 0:
            continue
        
        info = LABEL_DEFINITIONS[label]
        print(f"{label.upper()} ({info['name']} {info['range']}) - Total {len(label_data)} entries")
        print("-" * 80)
        
        # Randomly sample 3 entries
        samples = label_data.sample(min(3, len(label_data)))
        
        for idx, (i, row) in enumerate(samples.iterrows(), 1):
            print(f"\n  {idx}. {row['ticker']} | {str(row['date'])[:10]} | Change: {row['price_change']:+.2f}%")
            print(f"     {row['text'][:100]}")
            if len(row['text']) > 100:
                print(f"     ...")
        
        print("\n")

def save_data(df):
    """Save relabeled data"""
    print_section("💾 Step 5: Save data")
    
    # Keep original label, add new label_7class
    # Also create a clean version for training
    
    # Save full version (including all columns)
    full_output = OUTPUT_FILE.replace('.csv', '_full.csv')
    df.to_csv(full_output, index=False, encoding='utf-8-sig')
    print(f"✅ Full data saved: {full_output}")
    print(f"   Columns: {', '.join(df.columns)}")
    print()
    
    # Save training version (only necessary columns)
    training_columns = ['ticker', 'date', 'text', 'label_7class', 'price_change']
    df_training = df[training_columns].copy()
    df_training = df_training.rename(columns={'label_7class': 'label'})
    
    df_training.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
    print(f"✅ Training data saved: {OUTPUT_FILE}")
    print(f"   Columns: {', '.join(df_training.columns)}")
    print()
    
    # Show file size
    import os
    file_size = os.path.getsize(OUTPUT_FILE) / 1024
    print(f"   File size: {file_size:.2f} KB")
    print(f"   Number of entries: {len(df_training)}")
    print()

def create_visualization(df):
    """Create visualization charts (optional)"""
    print_section("📈 Step 6: Generate visualization (optional)")
    
    try:
        import matplotlib.pyplot as plt
        plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
        plt.rcParams['axes.unicode_minus'] = False
        
        # Create charts
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Chart 1: Label distribution
        label_counts = df['label_7class'].value_counts().reindex(LABELS_ORDER)
        colors = ['#d32f2f', '#f57c00', '#fbc02d', '#9e9e9e', '#8bc34a', '#4caf50', '#2e7d32']
        
        axes[0].bar(range(len(LABELS_ORDER)), label_counts.values, color=colors)
        axes[0].set_xticks(range(len(LABELS_ORDER)))
        axes[0].set_xticklabels([LABEL_DEFINITIONS[l]['name'] for l in LABELS_ORDER], rotation=45)
        axes[0].set_ylabel('Count')
        axes[0].set_title('7-Class Label Distribution')
        axes[0].grid(axis='y', alpha=0.3)
        
        # Add value labels
        for i, v in enumerate(label_counts.values):
            axes[0].text(i, v + 50, str(v), ha='center', va='bottom')
        
        # Chart 2: Price change distribution
        axes[1].hist(df['price_change'], bins=50, color='steelblue', alpha=0.7, edgecolor='black')
        axes[1].axvline(x=-5, color='red', linestyle='--', alpha=0.5, label='Big/Medium Loss')
        axes[1].axvline(x=-2, color='orange', linestyle='--', alpha=0.5, label='Medium/Small Loss')
        axes[1].axvline(x=-1, color='yellow', linestyle='--', alpha=0.5, label='Small Loss/Flat')
        axes[1].axvline(x=1, color='lightgreen', linestyle='--', alpha=0.5, label='Flat/Small Gain')
        axes[1].axvline(x=2, color='green', linestyle='--', alpha=0.5, label='Small/Medium Gain')
        axes[1].axvline(x=5, color='darkgreen', linestyle='--', alpha=0.5, label='Medium/Big Gain')
        axes[1].set_xlabel('Price Change (%)')
        axes[1].set_ylabel('Frequency')
        axes[1].set_title('Price Change Distribution with Classification Thresholds')
        axes[1].legend(fontsize=8)
        axes[1].grid(alpha=0.3)
        
        plt.tight_layout()
        
        # Save chart
        output_image = OUTPUT_FILE.replace('.csv', '_distribution.png')
        plt.savefig(output_image, dpi=150, bbox_inches='tight')
        print(f"✅ Visualization chart saved: {output_image}")
        print()
        
        plt.close()
        
    except Exception as e:
        print(f"⚠️ Visualization generation failed: {e}")
        print("   (This does not affect training, can be skipped)")
        print()

def main():
    """Main function"""
    print("\n" + "=" * 80)
    print("🎯 Relabel Training Data to 7-Class")
    print("=" * 80)
    print()
    print("Converting from 3-class (positive/negative/neutral)")
    print("to 7-class (crash/drop/small_drop/stable/small_rise/rise/surge)")
    print()
    print("-" * 80)
    print()
    
    # 1. Load and relabel
    df = load_and_relabel()
    if df is None: 
        return
    
    # 2. Analyze distribution
    analyze_distribution(df)
    
    # 3. Show samples
    show_samples(df)
    
    # 4. Save data
    save_data(df)
    
    # 5. Generate visualization
    create_visualization(df)
    
    # Complete
    print("=" * 80)
    print("✅ Step 1 Complete! Data relabeling successful!")
    print("=" * 80)
    print()
    print(f"Generated files:")
    print(f"  • {OUTPUT_FILE}")
    print(f"  • {OUTPUT_FILE.replace('.csv', '_full.csv')}")
    print()
    print("Next step:")
    print("  Run train_finbert_7class.py to start training the model")
    print()

if __name__ == "__main__":
    main()


🎯 重新标注训练数据为7分类

将把 3分类 (positive/negative/neutral)
转换为 7分类 (crash/drop/small_drop/stable/small_rise/rise/surge)

--------------------------------------------------------------------------------


📂 第1步：加载原始数据

✅ 数据加载成功: 8377 条

原始3分类标签分布:
------------------------------------------------------------
  positive  :  3020 条 ( 36.1%) ██████████████████
  negative  :  1983 条 ( 23.7%) ███████████
  neutral   :  3374 条 ( 40.3%) ████████████████████


🏷️  第2步：重新标注为7分类

标注规则:
------------------------------------------------------------
  crash        (大跌): < -5%
  drop         (中跌): -5% ~ -2%
  small_drop   (小跌): -2% ~ -1%
  stable       (震荡): -1% ~ 1%
  small_rise   (小涨): 1% ~ 2%
  rise         (中涨): 2% ~ 5%
  surge        (大涨): > 5%

开始标注...
✅ 标注完成: 8377 / 8377 条


📊 第3步：分析新标签分布

7分类标签统计:
--------------------------------------------------------------------------------
标签           中文       区间                    数量        百分比 可视化                           
-------------------------------------

## PART 2: Using FinBERT to Train a 7-Class Classification Model

In [ ]:
# train_finbert_7class.py
"""
Train FinBERT 7-Class Classification Model
Predict stock price movements based on news: Big Loss/Medium Loss/Small Loss/Flat/Small Gain/Medium Gain/Big Gain
"""

from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from datasets import Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, 
    precision_recall_fscore_support, 
    confusion_matrix,
    classification_report
)
from sklearn.utils.class_weight import compute_class_weight
import torch
import os
from datetime import datetime
import json

# ==================== Configuration ====================
MODEL_NAME = 'ProsusAI/finbert'
DATA_FILE = 'financial_data/training_data_7class.csv'
OUTPUT_DIR = 'finbert_7class_model'

# Training parameters - Optimized version
TRAIN_PARAMS = {
    'num_epochs': 10,           # 10 epochs is sufficient, avoiding overfitting
    'batch_size': 16,
    'learning_rate': 2e-5,      # Recommended learning rate for FinBERT
    'weight_decay': 0.01,
    'warmup_ratio': 0.1,
    'max_length': 128,
    'early_stopping_patience': 3  # Increased to 3 epochs patience, giving the model more chances
}

# 7 labels (in order)
LABELS_ORDER = ['crash', 'drop', 'small_drop', 'stable', 'small_rise', 'rise', 'surge']

# Label mapping
label2id = {label: i for i, label in enumerate(LABELS_ORDER)}
id2label = {i: label for i, label in enumerate(LABELS_ORDER)}

# Label names and ranges (for display)
LABEL_INFO = {
    'crash': {'name': 'Big Loss', 'range': '< -5%'},
    'drop': {'name': 'Medium Loss', 'range': '-5% ~ -2%'},
    'small_drop': {'name': 'Small Loss', 'range': '-2% ~ -1%'},
    'stable': {'name': 'Flat', 'range': '-1% ~ 1%'},
    'small_rise': {'name': 'Small Gain', 'range': '1% ~ 2%'},
    'rise': {'name': 'Medium Gain', 'range': '2% ~ 5%'},
    'surge': {'name': 'Big Gain', 'range': '> 5%'}
}

# ==================== Helper Functions ====================

def print_section(title):
    """Print section divider"""
    print("\n" + "=" * 80)
    print(title)
    print("=" * 80 + "\n")

def load_and_prepare_data():
    """Load and prepare data"""
    print_section("📂 Step 1: Load Data")
    
    # Read data
    if not os.path.exists(DATA_FILE):
        print(f"❌ Error: Cannot find file {DATA_FILE}")
        return None
    
    df = pd.read_csv(DATA_FILE, encoding='utf-8-sig')
    print(f"✅ Data loaded successfully: {len(df)} entries")
    
    # Check required columns
    required_cols = ['text', 'label']
    for col in required_cols:
        if col not in df.columns:
            print(f"❌ Error: Missing '{col}' column")
            return None
    
    # Remove null values
    before = len(df)
    df = df[df['text'].notna() & df['label'].notna()]
    df = df[df['text'].str.strip() != '']
    after = len(df)
    
    if before > after:
        print(f"⚠️ Removed {before - after} empty entries")
    
    # Convert labels to IDs
    df['label_id'] = df['label'].map(label2id)
    
    # Check labels
    if df['label_id'].isna().any():
        print("❌ Error: Unrecognized labels exist")
        unknown_labels = df[df['label_id'].isna()]['label'].unique()
        print(f"Unknown labels: {unknown_labels}")
        return None
    
    print(f"✅ Valid data: {len(df)} entries")
    print()
    
    # Display label distribution
    print("7-class label distribution:")
    print("-" * 80)
    print(f"{'Label':<12} {'Name':<12} {'Range':<15} {'Count':>8} {'Percentage':>10} {'Visualization':<30}")
    print("-" * 80)
    
    for label in LABELS_ORDER:
        count = len(df[df['label'] == label])
        percentage = count / len(df) * 100
        bar = "█" * int(percentage / 2)
        info = LABEL_INFO[label]
        print(f"{label:<12} {info['name']:<12} {info['range']:<15} {count:>8} {percentage:>9.1f}% {bar}")
    
    print()
    
    return df

def split_dataset(df):
    """Split dataset"""
    print_section("✂️ Step 2: Split Dataset")
    
    # 70% training, 15% validation, 15% test
    train_df, temp_df = train_test_split(
        df, 
        test_size=0.3, 
        random_state=42, 
        stratify=df['label_id']
    )
    
    val_df, test_df = train_test_split(
        temp_df, 
        test_size=0.5, 
        random_state=42, 
        stratify=temp_df['label_id']
    )
    
    print(f"Training set: {len(train_df)} entries (70%)")
    print(f"Validation set: {len(val_df)} entries (15%)")
    print(f"Test set: {len(test_df)} entries (15%)")
    print()
    
    # Check label distribution in each set
    print("Label distribution in each dataset:")
    print("-" * 80)
    
    for dataset_name, dataset_df in [('Training set', train_df), ('Validation set', val_df), ('Test set', test_df)]:
        print(f"{dataset_name}:")
        for label in LABELS_ORDER: 
            count = len(dataset_df[dataset_df['label'] == label])
            print(f"  {label}: {count}")
        print()
    
    return train_df, val_df, test_df

def compute_class_weights(train_df):
    """Compute class weights (handle imbalanced data)"""
    print_section("⚖️ Step 3: Compute Class Weights")
    
    # Compute class weights
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.array(LABELS_ORDER),
        y=train_df['label'].values
    )
    
    print("Class weights (for balancing imbalanced data):")
    print("-" * 60)
    for label, weight in zip(LABELS_ORDER, class_weights):
        print(f"  {label:<12} ({LABEL_INFO[label]['name']}): {weight:.4f}")
    print()
    
    # Convert to tensor
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)
    
    return class_weights_tensor

def create_datasets(train_df, val_df, test_df, tokenizer):
    """Create Hugging Face Dataset"""
    print_section("🔄 Step 4: Data Preprocessing")
    
    print("Converting to Dataset format...")
    
    # Keep only necessary columns
    train_dataset = Dataset.from_pandas(train_df[['text', 'label_id']])
    val_dataset = Dataset.from_pandas(val_df[['text', 'label_id']])
    test_dataset = Dataset.from_pandas(test_df[['text', 'label_id']])
    
    print("✅ Dataset created successfully")
    print()
    
    # Tokenize
    print("Tokenizing text...")
    
    def tokenize_function(examples):
        return tokenizer(
            examples['text'],
            padding='max_length',
            truncation=True,
            max_length=TRAIN_PARAMS['max_length']
        )
    
    train_dataset = train_dataset.map(tokenize_function, batched=True)
    val_dataset = val_dataset.map(tokenize_function, batched=True)
    test_dataset = test_dataset.map(tokenize_function, batched=True)
    
    print("✅ Tokenization complete")
    print()
    
    # Set format
    train_dataset = train_dataset.rename_column('label_id', 'labels')
    val_dataset = val_dataset.rename_column('label_id', 'labels')
    test_dataset = test_dataset.rename_column('label_id', 'labels')
    
    train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
    val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
    test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
    
    return train_dataset, val_dataset, test_dataset

def compute_metrics(pred):
    """Compute evaluation metrics"""
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    
    # Accuracy
    accuracy = accuracy_score(labels, preds)
    
    # Precision, Recall, F1
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted', zero_division=0
    )
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

class WeightedLossTrainer(Trainer):
    """Custom Trainer with class weight support"""
    
    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights
    
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        
        # Use weighted cross entropy loss
        if self.class_weights is not None:
            loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights.to(logits.device))
        else:
            loss_fct = torch.nn.CrossEntropyLoss()
        
        loss = loss_fct(logits, labels)
        
        return (loss, outputs) if return_outputs else loss

def train_model(train_dataset, val_dataset, tokenizer, class_weights):
    """Train model"""
    print_section("🚀 Step 5: Start Training")
    
    # Check device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    if device == "cuda":
        print(f"GPU: {torch.cuda.get_device_name(0)}")
    else:
        print("⚠️ Using CPU for training, speed will be slower (estimated 2-3 hours)")
        print("   If you have a GPU, please install the CUDA version of PyTorch")
    print()
    
    # Load model
    print("Loading FinBERT pretrained model...")
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=7,  # 7 classes
        id2label=id2label,
        label2id=label2id,
        ignore_mismatched_sizes=True
    )
    print("✅ Model loaded successfully")
    print(f"   Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    print()
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        num_train_epochs=TRAIN_PARAMS['num_epochs'],
        per_device_train_batch_size=TRAIN_PARAMS['batch_size'],
        per_device_eval_batch_size=TRAIN_PARAMS['batch_size'] * 2,
        learning_rate=TRAIN_PARAMS['learning_rate'],
        weight_decay=TRAIN_PARAMS['weight_decay'],
        warmup_ratio=TRAIN_PARAMS['warmup_ratio'],
        
        # Evaluation and save strategy
        eval_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
        metric_for_best_model='f1',
        greater_is_better=True,
        
        # Logging
        logging_dir=f'{OUTPUT_DIR}/logs',
        logging_steps=50,
        logging_strategy='steps',
        
        # Other
        push_to_hub=False,
        report_to='none',
        save_total_limit=2,
        fp16=torch.cuda.is_available(),  # Use mixed precision training if GPU available
    )
    
    # Create Trainer (with weighted loss)
    trainer = WeightedLossTrainer(
        class_weights=class_weights,
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=TRAIN_PARAMS['early_stopping_patience'])]
    )
    
    # Display training configuration
    print("Training configuration:")
    print("-" * 60)
    print(f"  Epochs: {TRAIN_PARAMS['num_epochs']}")
    print(f"  Batch size: {TRAIN_PARAMS['batch_size']}")
    print(f"  Learning rate: {TRAIN_PARAMS['learning_rate']}")
    print(f"  Total training steps: {len(train_dataset) // TRAIN_PARAMS['batch_size'] * TRAIN_PARAMS['num_epochs']}")
    print(f"  Using class weights: Yes")
    print(f"  Early stopping patience: {TRAIN_PARAMS['early_stopping_patience']} epochs")
    print()
    
    print("Starting training...")
    print("-" * 80)
    
    start_time = datetime.now()
    
    # Train
    trainer.train()
    
    end_time = datetime.now()
    duration = end_time - start_time
    
    print("-" * 80)
    print(f"✅ Training complete! Duration: {duration}")
    
    # Get training history
    if hasattr(trainer.state, 'log_history'):
        print(f"   Actual training epochs: {trainer.state.epoch}")
        if trainer.state.epoch < TRAIN_PARAMS['num_epochs']:
            print(f"   ⚠️ Early stopping triggered: stopped after epoch {int(trainer.state.epoch)} (max epochs set: {TRAIN_PARAMS['num_epochs']})")
    print()
    
    # Save model
    print("Saving model...")
    model.save_pretrained(OUTPUT_DIR)
    tokenizer.save_pretrained(OUTPUT_DIR)
    
    # Save label mapping
    label_mapping = {
        'label2id': label2id,
        'id2label': id2label,
        'label_info': LABEL_INFO,
        'labels_order': LABELS_ORDER
    }
    
    with open(f'{OUTPUT_DIR}/label_mapping.json', 'w', encoding='utf-8') as f:
        json.dump(label_mapping, f, ensure_ascii=False, indent=2)
    
    print(f"✅ Model saved to: {OUTPUT_DIR}")
    print(f"   - Model weights")
    print(f"   - Tokenizer")
    print(f"   - Label mapping")
    print()
    
    return trainer, model

def evaluate_model(trainer, test_dataset, test_df):
    """Evaluate model"""
    print_section("📊 Step 6: Evaluate Model")
    
    # Predict on test set
    print("Evaluating on test set...")
    predictions = trainer.predict(test_dataset)
    
    preds = np.argmax(predictions.predictions, axis=1)
    labels = predictions.label_ids
    
    # Compute metrics
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted', zero_division=0
    )
    
    print("Test set results:")
    print("-" * 60)
    print(f"  Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    print()
    
    # Detailed metrics for each class
    print("Detailed metrics for each class:")
    print("-" * 80)
    
    report = classification_report(
        labels, preds, 
        target_names=[f"{l} ({LABEL_INFO[l]['name']})" for l in LABELS_ORDER],
        digits=4,
        zero_division=0
    )
    print(report)
    print()
    
    # Confusion matrix
    print("Confusion matrix:")
    print("-" * 80)
    cm = confusion_matrix(labels, preds)
    
    # Pretty print
    print(f"{'Actual\\Predicted':>15}", end='')
    for label in LABELS_ORDER:
        print(f"{label:>10}", end='')
    print()
    print("-" * 80)
    
    for i, label in enumerate(LABELS_ORDER):
        print(f"{label:>15}", end='')
        for j in range(len(LABELS_ORDER)):
            print(f"{cm[i][j]:>10}", end='')
        print()
    print()
    
    # Error analysis
    print("Error sample analysis (showing first 5):")
    print("-" * 80)
    
    test_df_copy = test_df.copy().reset_index(drop=True)
    test_df_copy['predicted'] = [id2label[p] for p in preds]
    test_df_copy['correct'] = test_df_copy['label'] == test_df_copy['predicted']
    
    errors = test_df_copy[~test_df_copy['correct']]
    
    if len(errors) > 0:
        print(f"Total errors: {len(errors)} / {len(test_df_copy)} ({len(errors)/len(test_df_copy)*100:.1f}%)")
        print()
        
        for idx, row in errors.head(5).iterrows():
            print(f"\nError sample {idx + 1}:")
            print(f"  Text: {row['text'][:80]}...")
            print(f"  Actual label: {row['label']} ({LABEL_INFO[row['label']]['name']} {LABEL_INFO[row['label']]['range']})")
            print(f"  Predicted label: {row['predicted']} ({LABEL_INFO[row['predicted']]['name']} {LABEL_INFO[row['predicted']]['range']})")
            if 'price_change' in row:
                print(f"  Price change: {row['price_change']:.2f}%")
    else:
        print("🎉 No errors! Perfect!")
    
    print()
    
    return accuracy, f1, cm

def test_prediction(model, tokenizer):
    """Test prediction function"""
    print_section("🧪 Step 7: Test Prediction")
    
    # Get the device where the model is located
    device = next(model.parameters()).device
    print(f"Model on device: {device}")
    print()
    
    test_texts = [
        "Apple announces record-breaking quarterly earnings, stock soars 8%",
        "Tesla faces massive safety recall, shares plunge dramatically",
        "Microsoft reports steady cloud revenue growth",
        "NVIDIA unveils revolutionary AI chip, exceeds all expectations",
        "Amazon maintains market position amid competition",
        "Google announces minor organizational changes"
    ]
    
    print("Test news prediction:")
    print("-" * 80)
    
    for text in test_texts:
        # Tokenize
        inputs = tokenizer(
            text, 
            return_tensors='pt', 
            truncation=True, 
            max_length=128
        )
        
        # Move inputs to the device where the model is located
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # Predict
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            probs = torch.softmax(logits, dim=1)[0]
        
        pred_idx = torch.argmax(probs).item()
        pred_label = id2label[pred_idx]
        confidence = probs[pred_idx].item()
        
        # Display results
        print(f"\n📰 {text[:70]}...")
        print(f"   Prediction: {pred_label.upper()} ({LABEL_INFO[pred_label]['name']} {LABEL_INFO[pred_label]['range']})")
        print(f"   Confidence: {confidence:.2%}")
        
        # Display top 3 most likely classes
        top3_indices = torch.topk(probs, 3).indices
        print(f"   Top 3: ", end='')
        for idx in top3_indices:
            label = id2label[idx.item()]
            prob = probs[idx].item()
            print(f"{label}({prob:.1%}) ", end='')
        print()
    
    print()

def save_training_summary(accuracy, f1, cm, duration):
    """Save training summary"""
    summary = {
        'model_name': MODEL_NAME,
        'num_labels': 7,
        'labels': LABELS_ORDER,
        'training_params': TRAIN_PARAMS,
        'data_file': DATA_FILE,
        'test_accuracy': float(accuracy),
        'test_f1': float(f1),
        'training_duration': str(duration),
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    with open(f'{OUTPUT_DIR}/training_summary.json', 'w', encoding='utf-8') as f:
        json.dump(summary, f, ensure_ascii=False, indent=2)
    
    print(f"✅ Training summary saved: {OUTPUT_DIR}/training_summary.json")

def main():
    """Main function"""
    print("\n" + "=" * 80)
    print("🎯 FinBERT 7-Class Model Training")
    print("=" * 80)
    print(f"\nStart time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Data file: {DATA_FILE}")
    print(f"Output directory: {OUTPUT_DIR}")
    print()
    
    start_time = datetime.now()
    
    try:
        # 1. Load data
        df = load_and_prepare_data()
        if df is None:
            return
        
        # 2. Split dataset
        train_df, val_df, test_df = split_dataset(df)
        
        # 3. Compute class weights
        class_weights = compute_class_weights(train_df)
        
        # 4. Load tokenizer
        print_section("🔧 Load Tokenizer")
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        print("✅ Tokenizer loaded successfully")
        
        # 5. Create datasets
        train_dataset, val_dataset, test_dataset = create_datasets(
            train_df, val_df, test_df, tokenizer
        )
        
        # 6. Train model
        trainer, model = train_model(train_dataset, val_dataset, tokenizer, class_weights)
        
        # 7. Evaluate model
        accuracy, f1, cm = evaluate_model(trainer, test_dataset, test_df)
        
        # 8. Test prediction
        test_prediction(model, tokenizer)
        
        # 9. Save training summary
        end_time = datetime.now()
        duration = end_time - start_time
        save_training_summary(accuracy, f1, cm, duration)
        
        # Complete
        print_section("✅ Training Complete!")
        print(f"Final test accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
        print(f"Final F1-Score: {f1:.4f}")
        print(f"Model saved at: {OUTPUT_DIR}")
        print(f"Total duration: {duration}")
        print(f"\nEnd time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("=" * 80 + "\n")
        
        print("🎉 Congratulations! 7-class model training successful!")
        print()
        print("Next steps:")
        print("  1. Use the model to predict new news")
        print("  2. Analyze model performance")
        print("  3. Write project report")
        print()
        
    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()


🎯 FinBERT 7分类模型训练

开始时间: 2025-12-13 22:23:34
数据文件: financial_data/training_data_7class.csv
输出目录: finbert_7class_model


📂 第1步：加载数据

✅ 数据加载成功:  8377 条
✅ 有效数据: 8377 条

7分类标签分布:
--------------------------------------------------------------------------------
标签           中文     区间                    数量        百分比 可视化                           
--------------------------------------------------------------------------------
crash        大跌     < -5%                102       1.2% 
drop         中跌     -5% ~ -2%            880      10.5% █████
small_drop   小跌     -2% ~ -1%           1001      11.9% █████
stable       震荡     -1% ~ 1%            3374      40.3% ████████████████████
small_rise   小涨     1% ~ 2%             2018      24.1% ████████████
rise         中涨     2% ~ 5%              837      10.0% ████
surge        大涨     > 5%                 165       2.0% 


✂️ 第2步：划分数据集

训练集:  5863 条 (70%)
验证集: 1257 条 (15%)
测试集: 1257 条 (15%)

各数据集标签分布:
------------------------------------------------

Map:   0%|          | 0/5863 [00:00<?, ? examples/s]

Map:   0%|          | 0/1257 [00:00<?, ? examples/s]

Map:   0%|          | 0/1257 [00:00<?, ? examples/s]

✅ Tokenizing 完成


🚀 第5步：开始训练

使用设备: cuda
GPU:  NVIDIA GeForce RTX 4060 Laptop GPU

加载 FinBERT 预训练模型...


...

✅ 模型加载成功
   模型参数量: 109,487,623

训练配置:
------------------------------------------------------------
  Epochs: 10
  Batch size:  16
  Learning rate: 2e-05
  总训练步数: 3660
  使用类别权重: 是
  Early stopping patience: 3 轮

开始训练...
--------------------------------------------------------------------------------


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.918900,1.929901,0.185362,0.260759,0.185362,0.184609
2,1.719900,1.802483,0.185362,0.234727,0.185362,0.143664
3,1.650400,1.925867,0.199682,0.362848,0.199682,0.183674
4,1.345400,1.946019,0.221161,0.335865,0.221161,0.223619
5,1.125800,2.131968,0.245028,0.298651,0.245028,0.246421
6,0.963100,2.350612,0.247414,0.329535,0.247414,0.259660
7,0.845200,2.597356,0.272076,0.314367,0.272076,0.275644
8,0.665200,2.643265,0.303898,0.352034,0.303898,0.318622
9,0.586800,2.814600,0.303898,0.337569,0.303898,0.315045
10,0.480700,2.871447,0.308671,0.337108,0.308671,0.318426


--------------------------------------------------------------------------------
✅ 训练完成!  耗时: 0:07:54.040669
   实际训练轮数: 10.0

保存模型...
✅ 模型已保存到:  finbert_7class_model
   - 模型权重
   - Tokenizer
   - 标签映射


📊 第6步：评估模型

在测试集上评估...


测试集结果:
------------------------------------------------------------
  准确率 (Accuracy): 0.3023 (30.23%)
  精确率 (Precision): 0.3405
  召回率 (Recall): 0.3023
  F1-Score: 0.3156

各类别详细指标:
--------------------------------------------------------------------------------
                 precision    recall  f1-score   support

     crash (大跌)     0.3684    0.4375    0.4000        16
      drop (中跌)     0.1908    0.2500    0.2164       132
small_drop (小跌)     0.2000    0.2267    0.2125       150
    stable (震荡)     0.4881    0.3656    0.4181       506
small_rise (小涨)     0.3271    0.2871    0.3058       303
      rise (中涨)     0.1458    0.2240    0.1767       125
     surge (大涨)     0.1034    0.2400    0.1446        25

       accuracy                         0.3023      1257
      macro avg     0.2605    0.2901    0.2677      1257
   weighted avg     0.3405    0.3023    0.3156      1257


混淆矩阵:
--------------------------------------------------------------------------------
          真实\预测     c

Traceback (most recent call last):
  File "C:\Users\Ast\AppData\Local\Temp\ipykernel_6356\3697687283.py", line 610, in main
    print(f"最终测试准确率: {accuracy:. 4f} ({accuracy*100:.2f}%)")
                            ^^^^^^^^^^^^^^^
ValueError: Format specifier missing precision


## PART 3: Finbert 7-Class News Prediction Interaction

In [ ]:
# predict_news_stable.py

"""
Stable prediction script - Ensure reproducible results
"""
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import torch
import json
import random
import numpy as np

# ==================== Set Random Seed ====================

def set_seed(seed=42):
    """Set random seed to ensure reproducible results"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends. cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# ==================== Configuration ====================

MODEL_PATH = 'finbert_7class_model'

# Load label mapping

with open(f'{MODEL_PATH}/label_mapping.json', 'r', encoding='utf-8') as f:
    label_mapping = json.load(f)

LABELS_ORDER = label_mapping['labels_order']
id2label = {int(k): v for k, v in label_mapping['id2label'].items()}
LABEL_INFO = label_mapping['label_info']

# ==================== Load Model ====================

print("=" * 80)
print("🎯 FinBERT 7-Class News Prediction Tool (Stable Version)")
print("=" * 80)
print()
print("Loading model...")

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)

    # ✅ Set to evaluation mode (important!)
    model.eval()

    # Move to device

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model = model.to(device)
    print("✅ Model loaded successfully!")
    print(f"   Model path: {MODEL_PATH}")
    print(f"   Device:     {device}")
    print(f"   Mode:       Evaluation (eval)")
    print()
except Exception as e:
    print(f"❌ Failed to load model: {e}")
    exit()

# ==================== Prediction Function ====================

def predict_single_news(text):

    """Predict a single news item (stable version)"""
    
    # Tokenize
    inputs = tokenizer(
        text, 
        return_tensors='pt', 
        truncation=True, 
        max_length=128,
        padding=True
    )

    # Move to correct device

    inputs = {k: v.to(device) for k, v in inputs.items()}

    # ✅ Predict (disable gradient calculation)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch. softmax(logits, dim=1)[0].cpu()

    # Get prediction result

    predicted_idx = torch.argmax(probabilities).item()
    predicted_label = id2label[predicted_idx]
    confidence = probabilities[predicted_idx]. item()
    
    # Build result

    result = {
        'text': text,
        'predicted_label': predicted_label,
        'predicted_name': LABEL_INFO[predicted_label]['name'],
        'predicted_range': LABEL_INFO[predicted_label]['range'],
        'confidence': confidence,
        'all_probabilities': {
            LABELS_ORDER[i]: probabilities[i].item() 
            for i in range(len(LABELS_ORDER))
        }
    }
    
    return result

def display_prediction(result):

    """Nicely display prediction result"""

    print("=" * 80)
    print("📰 News:")
    print(f"   {result['text']}")
    print()
    print("🎯 Prediction Result:")
    print("-" * 80)
    print(f"   Predicted Label: {result['predicted_label'].upper()}")
    print(f"   Name:           {result['predicted_name']}")
    print(f"   Expected Range: {result['predicted_range']}")
    print(f"   Confidence:     {result['confidence']:.2%}")
    print()

    # Show all probabilities

    print("📊 Probabilities for Each Class:")

    print("-" * 80)
    sorted_probs = sorted(
        result['all_probabilities']. items(), 
        key=lambda x: x[1], 
        reverse=True
    )
    
    for label, prob in sorted_probs: 
        info = LABEL_INFO[label]
        bar = "█" * int(prob * 50)
        print(f"   {label. ljust(12)} ({info['name']} {info['range']}): {prob: >6.2%} {bar}")
    
    print("=" * 80)
    print()

# ==================== Consistency Test ====================

def test_consistency():
    """Test prediction consistency"""
    print("=" * 80)
    print("🧪 Consistency Test")
    print("=" * 80)
    print()
    
    test_text = "Apple announces record quarterly earnings beating all estimates"

    print(f"Test news: {test_text}")
    print()
    print("Predict 5 times in a row, check if results are consistent:")
    print("-" * 80)
    results = []
    for i in range(5):
        result = predict_single_news(test_text)
        results.append(result)
        print(f"Run {i+1}:  {result['predicted_label']} - {result['confidence']:.4f}")

    # Check consistency

    labels = [r['predicted_label'] for r in results]
    confidences = [r['confidence'] for r in results]
    
    print()
    if len(set(labels)) == 1:
        print("✅ Results are consistent! Prediction is stable.")
        print(f"   Std Dev: {np.std(confidences):.6f}")

    else:
        print("❌ Results are inconsistent!")
        print(f"   Predicted labels: {set(labels)}")
        print("   Possible reason: Model still has randomness or is overfitted")

    
    print()

# ==================== Main Program ====================

def main():
    print("Please select:")
    print("  1. Consistency Test")
    print("  2. Interactive Prediction")
    print("  3. Exit")
    print()

    choice = input("Select (1-3):  ").strip()

    print()
    
    if choice == '1':
        test_consistency()
    elif choice == '2':
        while True:
            text = input("📰 Please enter news (enter q to quit): ").strip()
            if text.lower() == 'q':
                break
            if text:
                result = predict_single_news(text)
                display_prediction(result)
    elif choice == '3':
        print("👋 Goodbye!")



if __name__ == "__main__":
    main()

🎯 FinBERT 7分类新闻预测工具（稳定版）

加载模型...
✅ 模型加载成功!
   模型位置: finbert_7class_model
   设备:   cuda
   模式: 评估模式 (eval)

请选择:
  1. 一致性测试
  2. 交互式预测
  3. 退出


📰 新闻:
   The situation of coated magazine printing paper will continue to be weak

🎯 预测结果:
--------------------------------------------------------------------------------
   预测类别: RISE
   中文名称: 中涨
   预期涨跌: 2% ~ 5%
   置信度:    43.29%

📊 各类别概率:
--------------------------------------------------------------------------------
   rise         (中涨 2% ~ 5%): 43.29% █████████████████████
   small_rise   (小涨 1% ~ 2%): 31.48% ███████████████
   stable       (震荡 -1% ~ 1%): 15.80% ███████
   drop         (中跌 -5% ~ -2%):  8.22% ████
   small_drop   (小跌 -2% ~ -1%):  0.82% 
   crash        (大跌 < -5%):  0.27% 
   surge        (大涨 > 5%):  0.11% 

📰 新闻:
   This implementation is very important to the operator , since it is about to launch its Fixed to Mobile convergence service in Brazil

🎯 预测结果:
-----------------------------------------------------------------

## PART 4: Finbert 7-Class News Prediction Visualization Interaction

In [1]:
# Add this cell at the bottom of your Jupyter Notebook

import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import torch

# Ensure the model is loaded (if not, load it first)
MODEL_PATH = 'finbert_7class_model'

if 'model' not in globals():
    print("🔄 Loading model...")
    from transformers import AutoTokenizer, AutoModelForSequenceClassification
    import json
    
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
    model.eval()
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    with open(f'{MODEL_PATH}/label_mapping.json', 'r', encoding='utf-8') as f:
        label_mapping = json.load(f)
    
    LABELS_ORDER = label_mapping['labels_order']
    id2label = {int(k): v for k, v in label_mapping['id2label'].items()}
    LABEL_INFO = label_mapping['label_info']
    
    print("✅ Model loaded!")

# Create interactive interface
print("=" * 80)
print("📰 FinBERT Financial News Sentiment Analysis - Interactive Prediction")
print("=" * 80)
print()

# Input box
text_input = widgets.Textarea(
    value='',
    placeholder='Enter news headline or content...',
    description='News: ',
    layout=widgets.Layout(width='100%', height='100px'),
    style={'description_width': '60px'}
)

# Predict button
predict_button = widgets.Button(
    description='🔍 Analyze',
    button_style='success',
    layout=widgets.Layout(width='150px', height='40px')
)

# Example buttons
example_buttons = []
examples = [
    "Apple announces record quarterly earnings beating all estimates, stock soars 10%",
    "Tesla faces massive safety recall affecting 2 million vehicles",
    "Microsoft reports steady cloud revenue growth in line with expectations",
    "NVIDIA unveils revolutionary AI chip exceeding analyst expectations"
]

for i, ex in enumerate(examples):
    btn = widgets.Button(
        description=f'Example {i+1}',
        button_style='info',
        layout=widgets.Layout(width='120px')
    )
    btn.example_text = ex
    example_buttons.append(btn)

# Output area
output_area = widgets.Output()

# Prediction function
def predict(text):
    """Predict and display result"""
    
    if not text or text.strip() == "":
        with output_area:
            clear_output()
            display(Markdown("⚠️ **Please enter news content**"))
        return
    
    with output_area:
        clear_output()
        display(Markdown("🔄 **Analyzing...**"))
    
    # Tokenize
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=128, padding=True)
    inputs = {k: v. to(device) for k, v in inputs.items()}
    
    # Predict
    with torch. no_grad():
        outputs = model(**inputs)
        probabilities = torch.softmax(outputs.logits, dim=1)[0].cpu().numpy()
    
    # Get result
    predicted_idx = probabilities.argmax()
    predicted_label = id2label[predicted_idx]
    confidence = probabilities[predicted_idx]
    
    info = LABEL_INFO[predicted_label]
    
    # Build display content
    result_md = f"""
---

### 📰 Input News
> {text[: 200]}{'...' if len(text) > 200 else ''}

---

### 🎯 Prediction Result

| Item | Result |
|------|------|
| **Predicted Label** | `{predicted_label.upper()}` |
| **Name** | {info['name']} |
| **Expected Range** | {info['range']} |
| **Confidence** | {confidence:.2%} |

---

### 💡 Investment Suggestion

"""
    
    # Add suggestion
    if confidence < 0.4:
        result_md += "⚠️ **Low confidence, consider waiting**\n"
    elif predicted_label in ['surge', 'rise']:
        if confidence > 0.7:
            result_md += "📈 **Strong bullish signal, consider buying**\n"
        else:
            result_md += "📈 **Bullish signal, buy with caution**\n"
    elif predicted_label in ['crash', 'drop']:
        if confidence > 0.7:
            result_md += "📉 **Strong bearish signal, consider selling or waiting**\n"
        else:
            result_md += "📉 **Bearish signal, be cautious**\n"
    elif predicted_label in ['small_rise']:
        result_md += "📊 **Slightly bullish, consider small position buying**\n"
    elif predicted_label in ['small_drop']:
        result_md += "📊 **Slightly bearish, consider reducing position**\n"
    else:
        result_md += "➡️ **Volatile signal, maintain position**\n"
    
    result_md += "\n> ⚠️ For reference only. Investment involves risk. Make decisions carefully.\n\n---\n\n"
    
    # Probability distribution
    result_md += "### 📊 Probability Distribution for Each Class\n\n"
    
    # Sort
    prob_pairs = [(LABELS_ORDER[i], probabilities[i]) for i in range(len(LABELS_ORDER))]
    prob_pairs.sort(key=lambda x: x[1], reverse=True)
    
    for label, prob in prob_pairs:
        info_label = LABEL_INFO[label]
        bar = "█" * int(prob * 30)
        result_md += f"- **{label}** ({info_label['name']} {info_label['range']}): {prob:.2%} {bar}\n"
    
    # Display result
    with output_area:
        clear_output()
        display(Markdown(result_md))

# Button click events
def on_predict_clicked(b):
    predict(text_input.value)

def on_example_clicked(b):
    text_input.value = b.example_text
    predict(b.example_text)

predict_button.on_click(on_predict_clicked)
for btn in example_buttons:
    btn.on_click(on_example_clicked)

# Layout
example_box = widgets.HBox(example_buttons)

ui = widgets.VBox([
    widgets.HTML("<h2>📝 Input News</h2>"),
    text_input,
    widgets.HBox([predict_button]),
    widgets.HTML("<h3>💡 Quick Examples</h3>"),
    example_box,
    widgets.HTML("<br>"),
    output_area
])

# Display interface
display(ui)

# Initial prompt
with output_area:
    display(Markdown("""
### 👋 Welcome to FinBERT Financial News Sentiment Analysis!

**How to use:**
1. Enter news in the input box above
2. Click the 🔍 Analyze button
3. Or click the example buttons for quick test

**7 Classes:**
- 🔴 Crash (<-5%)
- 🟠 Drop (-5~-2%)
- 🟡 Small Drop (-2~-1%)
- ⚪ Volatile (-1~1%)
- 🟢 Small Rise (1~2%)
- 🟢 Rise (2~5%)
- 🟢 Surge (>5%)
"""))

🔄 Loading model...
✅ Model loaded!
📰 FinBERT Financial News Sentiment Analysis - Interactive Prediction

